In [1]:
import os
import csv
from countries_languages import country_to_language

import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime

from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name


def remove_accents(text):
    return unidecode(text)

def get_country_code(country):
    matching_rows = countries_codes.loc[countries_codes['Country'] == country, ' Code']
    return matching_rows.iloc[0] if not matching_rows.empty else None

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

#PLAYER DATA - MARKET VALUE AND SALARY
leagues_salary = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1//CSVs we use often/latest_capology_data_money_fixed.csv')

leagues_value = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/MarketValuesComplete.csv')


leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')

#COUNTRY CODES
countries_codes = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/countries_and_codes.csv')

#FIFA RANKINGS
fifa_rankings = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/fifa_ranking-2023-10-26 - fifa_ranking-2023-07-20.csv')

#MATCH DATA - LINEUPS AND RESULTS
# lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_march_24.csv')
lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_april_29.csv')
results_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_results_data_march_24.csv')


leagues_value['Accents Removed'] = leagues_value['Name'].apply(lambda x: unidecode(x))
leagues_value['Team 1 Code'] = leagues_value['Nationality'].apply(get_country_code)

countries_codes['Accents Removed'] = countries_codes['Country'].apply(remove_accents)




/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/AssemblyHelpers.py:54: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)
/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_59433/2775728346.py:62: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')


In [3]:
#HELPER METHODS
##SELENIUM WRAPPER FUNCTION 

def useSeleniumToFindMoney(
        NAME_TO_FIND, #the name it is looking up
        NATIONALITY_INPUT_PLAYER, #natl_test
        DATE_INPUT_MATCH, #input_year_test
        SALARY_BOOLEAN):
    if(SALARY_BOOLEAN == False):
        print('finding salary with selenium')
        marketvalue = AssemblyHelpers.seleniumLookUpValueWrapperFunction(NAME_TO_FIND, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH)
        return marketvalue
    elif(SALARY_BOOLEAN == True):
        salary = AssemblyHelpers.capology_selenium_lookup(NAME_TO_FIND, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH)
        return salary

    else:
        return 0



#MATCH DATE FILTER 
def filter_using_date_of_match(candidate_list_of_names_input, nationality_input, matchdate_Input, SALARY_BOOLEAN): 

    FILTERED_NAMES_USING_MATCH_DATE = AssemblyHelpers.multiNameMatchDateLookup(candidate_list_of_names_input, nationality_input, matchdate_Input) 

    print('in fudom', type(FILTERED_NAMES_USING_MATCH_DATE))
    #1 MATCH
    if isinstance(FILTERED_NAMES_USING_MATCH_DATE, str) or isinstance(FILTERED_NAMES_USING_MATCH_DATE, np.str_): #YELLOW 3
        print('were here', FILTERED_NAMES_USING_MATCH_DATE)
        #print('were in fudom in the right place')
        #NAME_FOR_SELENIUM_SEARCH = FILTERED_NAMES_USING_MATCH_DATE
        # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, nationality_input, matchdate_Input, SALARY_BOOLEAN)
        # print('made it out of using selenium')
        # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
        money_thisplayer = [0, 'Lookup Required', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #MULTIPLE MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) >= 2):
        print(FILTERED_NAMES_USING_MATCH_DATE, type(FILTERED_NAMES_USING_MATCH_DATE), type(FILTERED_NAMES_USING_MATCH_DATE) == str)
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {FILTERED_NAMES_USING_MATCH_DATE}', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #0 MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) == 0):
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, CANDIDATE NAMES WERE {candidate_list_of_names_input}', True, False]
        return money_thisplayer, ''



#FINDING MONEY INFO USSING PLAYER NAME
def tryToFindPlayerMoney(
        DATABASE_RETURN_MESSAGE, #what the DB said when trying to find money info
        NAME_TO_FIND, #the name it found in the DB (name_match[0] originally)
        NATIONALITY_INPUT_PLAYER, #natl_test
        DATE_INPUT_MATCH, #input_year_test
        SALARY_BOOLEAN, #IF TRUE, SALARIES. OTHERWISE VALUES
        ORIGINAL_SEARCH_NAME #name_match[2]
        ):

    if((DATABASE_RETURN_MESSAGE == "not in DB any of 3 seasons.") or (DATABASE_RETURN_MESSAGE == "asswipe was in DB in before or after season")):
        print('newwwnewnenwenewnwenwenwew', DATABASE_RETURN_MESSAGE)
        
        if DATABASE_RETURN_MESSAGE == 'asswipe was in DB in before or after season':
            money_thisplayer = [0, 'Was in  DB before or after season', True, True]
            return money_thisplayer

        
        
        

        INTERNET_LOOKUP_RESULTING_NAMES = AssemblyHelpers.false_name_match_lookup(ORIGINAL_SEARCH_NAME, NATIONALITY_INPUT_PLAYER)
        NUM_NAMES_REMAINING = len(list(INTERNET_LOOKUP_RESULTING_NAMES))
        print(INTERNET_LOOKUP_RESULTING_NAMES)
        #1 MATCH
        if(NUM_NAMES_REMAINING == 1):
            #FIND INFO USING SELENIUM
            # NAME_FOR_SELENIUM_SEARCH = next(iter(INTERNET_LOOKUP_RESULTING_NAMES))
            # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH, SALARY_BOOLEAN)
            # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
            money_thisplayer = [0, 'Lookup Required', True, False]
            return money_thisplayer
        #MULTIPLE MATCHES
        elif(NUM_NAMES_REMAINING >= 2):
            print('goes to FUDOM')
            #FILTER THE NAMES USING THE DATE OF THE MATCH
            list_to_filter_with_match_date = list(INTERNET_LOOKUP_RESULTING_NAMES)
            #NEW FUNCTION 3 
            money_thisplayer, name_results = filter_using_date_of_match(list_to_filter_with_match_date, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH, SALARY_BOOLEAN) 
            return money_thisplayer
        #0 MATCHES
        elif(NUM_NAMES_REMAINING == 0):
            #This is return case 9
            money_thisplayer = [0, 'Online Lookup Required', True, False] 
            return money_thisplayer
    
    else:
        money_thisplayer = [0, 'Online Lookup Required', True, False]
        return money_thisplayer
    


    #old
    # elif(DATABASE_RETURN_MESSAGE == "was in DB in before or after season"):
    #     0==0
    

##OVERARCHING Controller FUNCTION 
def new_overarching(starter_number, name_input, nationality_input, date_input, salary_boolean, competition_input, combination_dict_matches, combination_dict_NON_matches, combination_dict_multi_matches):
    """
    Inputs: Name, Date, Nationality, Salary Boolean 
        salary_boolean
            true – salary
            false – Market Value

    Outputs: Money_thisplayer, name_match
    Outputs to add: error, error type, error msg
    """

    #SETTING INITIAL VALUES
    example_problem = name_input
    natl_test = nationality_input
    input_year_test = date_input

    if 'U-20' in natl_test:
        print('u-20 team')
        natl_test = natl_test.split(' U-20')[0]

    corrrect_code = countries_codes[countries_codes['Country'] == natl_test][' Code'].unique()[0]
    

    if salary_boolean == True:
        0==0
        #dataset_nationality = leagues_salary[leagues_salary['Team 1 Code'] == corrrect_code]['Name'].unique()
    else:
        dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == corrrect_code]['Name'].unique()

    money_thisplayer = [0, 'not done yet', True, False]
    secondmatch_required = False

    #FIRST TRY LOOKING FOR NAME
    name_match = AssemblyHelpers.lookup_name(example_problem, natl_test, input_year_test, salary_boolean, competition_input)
    
    match_case = name_match[1]
    original_db_name_for_search = name_match[2]
    print('match is', name_match)

    

    if(match_case == 'single'):
        NAME_MATCH_FIRST_TRY = name_match[0]
        #print(NAME_MATCH_FIRST_TRY)
        money_thisplayer = AssemblyHelpers.find_money_info_from_name(NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values)

        money_found = money_thisplayer[0]
        search_db_return_case = money_thisplayer[1]
        online_lookup_required = money_thisplayer[2]

        if ((money_found == 0) & (online_lookup_required == True)):         #call the find_money_info_extended once you write it. inputs are = (NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, name_match[2])
            #NEW FUNCTION 1
            if '.' in input_year_test:
                yearstr = input_year_test.split(".")[2]
                full_num = int('20' + yearstr)
            elif '-' in input_year_test:
                if len(input_year_test.split("-")[2]) == 4:
                    yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                    full_num = int(yearstr)
                elif len(input_year_test.split("-")[2]) == 2:
                    if len(input_year_test.split("-")[0]) == 4:
                        yearstr = input_year_test.split("-")[0]
                        full_num = int(yearstr)
                    else:
                        yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                        full_num = int('20' + yearstr)
                else:
                    print('broke on year parsing', input_year_test.split("-"))
                # elif len(input_year_test.split("-")[0]) == 4:
                    # yearstr = input_year_test.split("-")[0]
                    # full_num = yearstr
            
            #print(f'singi trying {NAME_MATCH_FIRST_TRY} and {int('20' + yearstr)}')
            player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num)].reset_index()
            
            if len(player_bigvalue_dataset) >= 1:
                #print(player_bigvalue_dataset.at[0, 'Market Value'])
                #money_thisplayer[0] =
                #print('working youre good')
                money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                        #money_thisplayer[2] = True
                        money_thisplayer[1] = 'Large Dataset had 0'
            else:
                if len(leagues_value_large[((leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num - 1)) | 
                                       ((leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num+ 1))]) >= 2:
                    money_thisplayer = [0, 'Not in either dataset for currect season but present in both seasons around match', False, True]
                    # leagues_value_large[((leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num - 1)) | 
                    #                    ((leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num + 1))]['Market Value'].mean()
                else:
                    if ((leagues_value[leagues_value['Name'] == NAME_MATCH_FIRST_TRY]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY]['Season'].max() == (full_num - 1))):
                        money_last_season = leagues_value[(leagues_value['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                        money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                    else:
                        money_thisplayer = tryToFindPlayerMoney(search_db_return_case, NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
            
        elif(search_db_return_case == 'working'):
            0==0
            #return to the end basically
        elif(search_db_return_case == 'working - imputed'):
            0==0
            #when ur done this should be removed. go into the function find_money_info and remove case
                    
    elif(match_case == 'multiple'):
        print('multi match')

        if '.' in input_year_test:
            yearstr = input_year_test.split(".")[2]
            full_num = int('20' + yearstr)
        elif '-' in input_year_test:
            if len(input_year_test.split("-")[2]) == 4:
                yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                full_num = int(yearstr)
            elif len(input_year_test.split("-")[2]) == 2:
                if len(input_year_test.split("-")[0]) == 4:
                    yearstr = input_year_test.split("-")[0]
                    full_num = int(yearstr)
                else:
                    yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                    full_num = int('20' + yearstr)

        combination_key = (example_problem, full_num)

        if combination_key in combination_dict_matches:

            print('combo is in matches dict')
            #correct_name = combination_dict_matches[combination_key]
            #print('correct name is ', correct_name)

            SINGLE_NAME_MATCH_CORRECT_SEASON = combination_dict_matches[combination_key]
                
            money_thisplayer = AssemblyHelpers.find_money_info_from_name(SINGLE_NAME_MATCH_CORRECT_SEASON[0], natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 

            money_found = money_thisplayer[0]
            search_db_return_case = money_thisplayer[1]
            online_lookup_required = money_thisplayer[2]

            if ((money_found == 0) & (online_lookup_required == True)):  
                #NEW FUNCTION 1     
                if '.' in input_year_test:
                    yearstr = input_year_test.split(".")[2]
                    full_num = int('20' + yearstr)
                elif '-' in input_year_test:
                    if len(input_year_test.split("-")[2]) == 4:
                        yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                        full_num = int(yearstr)
                    elif len(input_year_test.split("-")[2]) == 2:
                        if len(input_year_test.split("-")[0]) == 4:
                            yearstr = input_year_test.split("-")[0]
                            full_num = int(yearstr)
                        else:
                            yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                            full_num = int('20' + yearstr)
                    else:
                        print('broke on year parsing', input_year_test.split("-"))
        
                #print(f'multiii trying {SINGLE_NAME_MATCH_CORRECT_SEASON} and {int('20' + yearstr)}')
                if type(SINGLE_NAME_MATCH_CORRECT_SEASON) == list:
                    SINGLE_NAME_MATCH_CORRECT_SEASON = SINGLE_NAME_MATCH_CORRECT_SEASON[0]
                player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value_large['Season'] == full_num)].reset_index()
                
                if len(player_bigvalue_dataset) >= 1:
                    #print(player_bigvalue_dataset.at[0, 'Market Value'])
                    #money_thisplayer[0] =
                    #print('working youre good')
                    money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                    if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                        #money_thisplayer[2] = True
                        money_thisplayer[1] = 'Large Dataset said 0'
                else:
                    if ((leagues_value[leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1))):
                        money_last_season = leagues_value[(leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                        money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                    else:
                        money_thisplayer = tryToFindPlayerMoney(search_db_return_case, SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                        
            elif(search_db_return_case == 'working'):
                0==0
                #return to the end basically
            elif(search_db_return_case == 'working - imputed'):
                0==0
                #when ur done this should be removed. go into the function find_money_info and remove case
            
            #in this situation, you need to do find money info from name. 
            #use the name that is in the dict 
        elif combination_key in combination_dict_NON_matches:
            print('combination in non-matches dict')#if you find it in the non matches dictionary
            #save the list of names it returned 
            #then return money thisplayer 
            NAMES_MATCHES_LIST_CORRECT_SEASON = combination_dict_NON_matches[combination_key]
            money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, ORIGINAL NAME WAS {example_problem}', True, False]
        elif combination_key in combination_dict_multi_matches:
            #stop the loop here 
            filtered_names_match_date = combination_dict_multi_matches[combination_key]
            money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {filtered_names_match_date}', True, False]
        else:
            print('combo is not in matches, multi-matches or non-matches dict')
        
            NAMES_MATCHES_LIST = name_match[0]
            
            
            #FILTER THE NAMES USING THE YEAR OF THE MATCH
            NAMES_MATCHES_LIST_CORRECT_SEASON = AssemblyHelpers.filter_candidates_using_year(starter_number, NAMES_MATCHES_LIST, natl_test, input_year_test, salary_boolean)
            

            NUMBER_OF_NAMES_MATCHING = len(NAMES_MATCHES_LIST_CORRECT_SEASON)
            #print(f'left is {NUMBER_OF_NAMES_MATCHING}')
            
            if(NUMBER_OF_NAMES_MATCHING == 1): #YELLOW 2
                
                
                SINGLE_NAME_MATCH_CORRECT_SEASON = NAMES_MATCHES_LIST_CORRECT_SEASON[0]
                
                money_thisplayer = AssemblyHelpers.find_money_info_from_name(SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 

                money_found = money_thisplayer[0]
                search_db_return_case = money_thisplayer[1]
                online_lookup_required = money_thisplayer[2]

                if ((money_found == 0) & (online_lookup_required == True)):  
                    #NEW FUNCTION 1     
                    if '.' in input_year_test:
                        yearstr = input_year_test.split(".")[2]
                        full_num = int('20' + yearstr)
                    elif '-' in input_year_test:
                        if len(input_year_test.split("-")[2]) == 4:
                            yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                            full_num = int(yearstr)
                        elif len(input_year_test.split("-")[2]) == 2:
                            if len(input_year_test.split("-")[0]) == 4:
                                yearstr = input_year_test.split("-")[0]
                                full_num = int(yearstr)
                            else:
                                yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                                full_num = int('20' + yearstr)
                        else:
                            print('broke on year parsing', input_year_test.split("-"))
            
                    #print(f'multiii trying {SINGLE_NAME_MATCH_CORRECT_SEASON} and {int('20' + yearstr)}')
                    print(full_num, SINGLE_NAME_MATCH_CORRECT_SEASON)
                    player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value_large['Season'] == full_num)].reset_index()
                    
                    if len(player_bigvalue_dataset) >= 1:
                        #print(player_bigvalue_dataset.at[0, 'Market Value'])
                        #money_thisplayer[0] =
                        #print('working youre good')
                        money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                        if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                            #money_thisplayer[2] = True
                            money_thisplayer[1] = 'Large Dataset said 0'
                    else:
                        if ((leagues_value[leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1))):
                            money_last_season = leagues_value[(leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                            money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                        else:
                            money_thisplayer = tryToFindPlayerMoney(search_db_return_case, SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                            
                elif(search_db_return_case == 'working'):
                    0==0
                    #return to the end basically
                elif(search_db_return_case == 'working - imputed'):
                    0==0
                    #when ur done this should be removed. go into the function find_money_info and remove case
                
            elif(NUMBER_OF_NAMES_MATCHING >= 2):
                #use match date to filter 
                #print('list of names is ', NAMES_MATCHES_LIST_CORRECT_SEASON)
                #print('we are here', NUMBER_OF_NAMES_MATCHING)
                if('Olympics' in competition_input):
                    money_thisplayer = [0, F'Olympics row. Couldn\'t filter names using Transfermarkt. remaining matches were {NAMES_MATCHES_LIST_CORRECT_SEASON}', True, False]
                    #the filtering is not going to work 
                else:
                
                    filtered_names_match_date = AssemblyHelpers.multiNameMatchDateLookup(NAMES_MATCHES_LIST_CORRECT_SEASON, natl_test, input_year_test) 

                    #1 MATCH
                    if(type(filtered_names_match_date) == str): #YELLOW 3
                        
                        if '.' in input_year_test:
                            yearstr = input_year_test.split(".")[2]
                            full_num = int('20' + yearstr)
                        elif '-' in input_year_test:
                            if len(input_year_test.split("-")[2]) == 4:
                                yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                                full_num = int(yearstr)
                            elif len(input_year_test.split("-")[2]) == 2:
                                if len(input_year_test.split("-")[0]) == 4:
                                    yearstr = input_year_test.split("-")[0]
                                    full_num = int(yearstr)
                                else:
                                    yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                                    full_num = int('20' + yearstr)
                        #print('we here', filtered_names_match_date, full_num, example_problem)
                        combination_key = (example_problem, full_num)
                        combination_dict_matches[combination_key] = [filtered_names_match_date]
                        print(f'added key {combination_key} to dict. value is {filtered_names_match_date}')
                        #at this point, example problem and full_num become a combination 
                        #that combination should be a key, and the value should be filtered_names_match_date
                        #add a new key+value combination to the dictionary here. 

                        NAME_MATCH_SINGLE = filtered_names_match_date

                        #name_match[0] = NAME_MATCH_SINGLE
                        #print('we hereeeee', NAME_MATCH_SINGLE)
                        money_thisplayer = AssemblyHelpers.find_money_info_from_name(NAME_MATCH_SINGLE, natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 
                        
                        money_found = money_thisplayer[0]
                        search_db_return_case = money_thisplayer[1]
                        online_lookup_required = money_thisplayer[2]

                        if ((money_found == 0) & (online_lookup_required == True)):         #call the find_money_info_extended once you write it. inputs are = (NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, name_match[2])
                            # NAME_FOR_SELENIUM_SEARCH = NAME_MATCH_SINGLE
                            # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, natl_test, input_year_test, salary_boolean)
                            # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {salary_boolean}', True, False]

                            if len(leagues_value_large[(leagues_value_large['Name'] == NAME_MATCH_SINGLE) & (leagues_value_large['Season'] == full_num)]) >= 1:
                                money_thatseason_large_db = leagues_value_large[(leagues_value_large['Name'] == NAME_MATCH_SINGLE) & (leagues_value_large['Season'] == full_num)]['Market Value'].max()
                                money_thisplayer = [money_thatseason_large_db, 'Used Large Dataset', False, False]

                            elif ((leagues_value[leagues_value['Name'] == NAME_MATCH_SINGLE]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == NAME_MATCH_SINGLE]['Season'].max() == (full_num - 1))):
                                money_last_season = leagues_value[(leagues_value['Name'] == NAME_MATCH_SINGLE) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                                money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                            else:
                                #money_thisplayer = tryToFindPlayerMoney(search_db_return_case, NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                                money_thisplayer = [0, 'Lookup Required', True, False]
                            #return money_thisplayer
                            
                        elif(search_db_return_case == 'working'):
                            0==0
                            #return to the end basically
                        elif(search_db_return_case == 'working - imputed'):
                            0==0
                            #when ur done this should be removed. go into the function find_money_info and remove case


                    #MULTIPLE MATCHES
                    elif(len(filtered_names_match_date) >= 2): #RED 6

                        print('added a row to multi matches dictionary.')
                        combination_key_multi_match = (example_problem, full_num)
                        combination_dict_multi_matches[combination_key_multi_match] = [filtered_names_match_date]
                        
                        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {filtered_names_match_date}', True, False]
                        #NNNNEW - add to the dictionary here 
                        #return money_thisplayer, name_match
                    #0 MATCHES
                    elif(len(filtered_names_match_date) == 0): #RED 5
                        #print('we are here. bahrain case')
                        if '.' in input_year_test:
                            yearstr = input_year_test.split(".")[2]
                            full_num = int('20' + yearstr)
                        elif '-' in input_year_test:
                            if len(input_year_test.split("-")[2]) == 4:
                                yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                                full_num = int(yearstr)
                            elif len(input_year_test.split("-")[2]) == 2:
                                if len(input_year_test.split("-")[0]) == 4:
                                    yearstr = input_year_test.split("-")[0]
                                    full_num = int(yearstr)
                                else:
                                    yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                                    full_num = int('20' + yearstr)

                        combination_key_nomatch = (example_problem, full_num)
                        combination_dict_NON_matches[combination_key_nomatch] = [NAMES_MATCHES_LIST_CORRECT_SEASON]
                        print(f'added a row to non matches dict. player was {example_problem} in {full_num}.')
                        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, ORIGINAL NAME WAS {example_problem}', True, False]
                        #return money_thisplayer, name_match
                    
            elif(NUMBER_OF_NAMES_MATCHING == 0):#RETURN CASE 4

                print('here rc4')
                
                money_thisplayer, name_results = filter_using_date_of_match(NAMES_MATCHES_LIST, natl_test, input_year_test, salary_boolean) 
                if money_thisplayer == [0, 'Lookup Required', True, False]:
                    SINGLE_NAME_MATCH_CORRECT_SEASON = name_results
                    combination_key = (example_problem, full_num)
                    combination_dict_matches[combination_key] = [SINGLE_NAME_MATCH_CORRECT_SEASON]
                    print(f'added key {combination_key} to dict. value is {SINGLE_NAME_MATCH_CORRECT_SEASON}')
                    player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value_large['Season'] == full_num)].reset_index()
                    
                    if len(player_bigvalue_dataset) >= 1:
                        #print(player_bigvalue_dataset.at[0, 'Market Value'])
                        #money_thisplayer[0] =
                        #print('working youre good')
                        money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                        if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                            #money_thisplayer[2] = True
                            money_thisplayer[1] = 'Large Dataset said 0'
                    else:
                        if ((leagues_value[leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1))):
                            money_last_season = leagues_value[(leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                            money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                        else:
                            money_thisplayer = tryToFindPlayerMoney(money_thisplayer[1], SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                
    

    elif(match_case == 'none'):
        #secondmatch_required = True
        #print('ya 1')

        NAME_TO_SEARCH = name_match[2]

        NAME_FOURTH_TRY = AssemblyHelpers.fourth_try_name_search(NAME_TO_SEARCH, natl_test, salary_boolean)

        MATCH_CASE_FOURTH_TRY = NAME_FOURTH_TRY[0]

        if(MATCH_CASE_FOURTH_TRY == 'single'): #YELLOW 1
            NAME_FOUND = NAME_FOURTH_TRY[1]
            money_thisplayer = AssemblyHelpers.find_money_info_from_name(NAME_FOUND, natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 

            money_found = money_thisplayer[0]
            search_db_return_case = money_thisplayer[1]
            online_lookup_required = money_thisplayer[2]

            if '.' in input_year_test:
                yearstr = input_year_test.split(".")[2]
                full_num = int('20' + yearstr)
            elif '-' in input_year_test:
                if len(input_year_test.split("-")[2]) == 4:
                    yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                    full_num = int(yearstr)
                elif len(input_year_test.split("-")[2]) == 2:
                    if len(input_year_test.split("-")[0]) == 4:
                        yearstr = input_year_test.split("-")[0]
                        full_num = int(yearstr)
                    else:
                        yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                        full_num = int('20' + yearstr)

            if ((money_found == 0) & (online_lookup_required == True)): #call the find_money_info_extended once you write it. inputs are = (NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, name_match[2])
                if ((leagues_value[leagues_value['Name'] == NAME_FOUND]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == NAME_FOUND]['Season'].max() == (full_num - 1))):
                    money_last_season = leagues_value[(leagues_value['Name'] == NAME_FOUND) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                    money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                else:
                    money_thisplayer = tryToFindPlayerMoney(search_db_return_case, NAME_FOUND, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                
            elif(search_db_return_case == 'working'):
                0==0
                #return to the end basically
            elif(search_db_return_case == 'working - imputed'):
                0==0
                #when ur done this should be removed. go into the function find_money_info and remove case

        elif(MATCH_CASE_FOURTH_TRY == 'multiple'):#RED 3

            print('here 4th try')
    
            LIST_TO_FILTER = NAME_FOURTH_TRY[1]
            money_thisplayer, name_results = filter_using_date_of_match(LIST_TO_FILTER, natl_test, input_year_test, salary_boolean) 
            if money_thisplayer == [0, 'Lookup Required', True, False]:
                SINGLE_NAME_MATCH_CORRECT_SEASON = name_results
                combination_key = (example_problem, full_num)
                combination_dict_matches[combination_key] = [SINGLE_NAME_MATCH_CORRECT_SEASON]
                print(f'added key {combination_key} to dict. value is {SINGLE_NAME_MATCH_CORRECT_SEASON}')
                player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value_large['Season'] == full_num)].reset_index()
                
                if len(player_bigvalue_dataset) >= 1:
                    #print(player_bigvalue_dataset.at[0, 'Market Value'])
                    #money_thisplayer[0] =
                    #print('working youre good')
                    money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                    if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                        #money_thisplayer[2] = True
                        money_thisplayer[1] = 'Large Dataset said 0'
                else:
                    if ((leagues_value[leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1))):
                        money_last_season = leagues_value[(leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                        money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                    else:
                        money_thisplayer = tryToFindPlayerMoney(money_thisplayer[1], SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
        
        elif(MATCH_CASE_FOURTH_TRY == 'none'): #SELENIUM
            #print('ya 2')
            original_db_name_for_search = name_match[2]

            accents_removed_name = remove_accents(original_db_name_for_search)
            #print(accents_removed_name)
            equal_names = [name for name in dataset_nationality if name == accents_removed_name]
            #(accents_removed_name in dataset_nationality)

            if equal_names:
                0==0
            else:
            #print
                matching_names = [name for name in dataset_nationality if name.endswith(accents_removed_name)]
                #print('weere here', matching_names)
                if len(matching_names) >= 2:
                    0==0 #more than 1 person remains
                elif len(matching_names) == 1:
                    0==0 #single matching name
                    #print('heylo')
                    money_thisplayer = AssemblyHelpers.find_money_info_from_name(matching_names[0], natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 
                else:
                    0==0 #no matches 
                    #print('nerr')
                    # MONEY_FOUND_ONLINE_0 = useSeleniumToFindMoney(original_db_name_for_search, natl_test, input_year_test, salary_boolean)
                    # money_thisplayer = [MONEY_FOUND_ONLINE_0, f'Online WAS Lookup Required - salary boolean is {salary_boolean}', True, False]
                    money_thisplayer = [0, 'Lookup Required', True, False]
    else:
        0==0 #RED 7

    return money_thisplayer, name_match


def remove_prefix(string):
    # Define pattern to match digits or '-' at the start of the string
    pattern = r'^[\d-]+'
    
    # Use re.sub to remove the matched pattern from the start of the string
    return re.sub(pattern, '', string)

In [2]:
combined_df = load_csv_dataset('combined_DF_terrikon_removed_IDs_FIXED.csv')

failed_df = load_csv_dataset('failed_df_terrikon_removed_IDs_FIXED.csv')

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_59433/2775728346.py:54: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [4]:
len(combined_df)

56928

In [5]:
len(failed_df)

427

In [90]:
def add_correct_money(money_column):
    if money_column == 0:
        return 0

    if 'm' in money_column:
        value_rough = money_column.split('m')[0].split('€')[1]
        millinum = (float(value_rough) * 1000000)
        return millinum
    elif 'k' in money_column:
        #print(i, 'k')
        value_rough = money_column.split('k')[0].split('€')[1]
        millinum = (float(value_rough) * 1000)

        return millinum
    elif '-' in money_column:
        return 0
    else:
        return 0

def in_season_around(player_id, curr_season, age):
    if age == "Not Listed": age = 30
    age = int(age)
    year_before = int(curr_season) -1
    year_after = int(curr_season) + 1
    before_season = leagues_value[(leagues_value['PlayerID'] == player_id) & (leagues_value['Season'] == year_before)] #leagues_value_large #leagues_value
    after_season =  leagues_value[(leagues_value['PlayerID'] == player_id) & (leagues_value['Season'] == year_after)]
    if before_season.empty and after_season.empty:
        return -1, "Both Empty"
    if not before_season.empty and after_season.empty:
        if before_season.iloc[0]['Market Value'] != "-":
            if before_season.iloc[0]['Market Value'] == 0:
                return 0, "Only Before + Zero Before"
            else:
                if age >= 30:
                    return (float(add_correct_money(before_season.iloc[0]['Market Value'])) * .8), "Only Before" #add_correct_money
                else:
                    return (float(add_correct_money(before_season.iloc[0]['Market Value'])) * 1.2), "Only Before" #add_correct_money
        else:
            return 0, "Only Before X"
    if not after_season.empty and before_season.empty:
        
        if after_season.iloc[0]['Market Value'] != "-":
            if after_season.iloc[0]['Market Value'] == 0:
                return 0, "Only After + Zero After"
            else:
                if age <= 30:
                    return (float(add_correct_money(after_season.iloc[0]['Market Value'])) * .8), "Only After"
                else:
                    return (float(add_correct_money(after_season.iloc[0]['Market Value'])) * 1.2), "Only After"
        else:
            return 0, "Only After X"
    if not before_season.empty and not after_season.empty:
        #print('hi')
        before_season_val = before_season.iloc[0]['Market Value']
        after_season_val = after_season.iloc[0]['Market Value']
        #print(before_season_val, after_season_val)
        if before_season_val != "-" and after_season_val != "-":
            # if type(before_season_val) == str:
            #     before_season_val = add_correct_money(before_season_val)
            # if type(after_season_val) == str:
            #     after_season_val = add_correct_money(after_season_val)
            return ((float(add_correct_money(before_season.iloc[0]['Market Value'])) + float(add_correct_money(after_season.iloc[0]['Market Value'])) / 2)), "Used Average of Season Before and After"
        elif before_season_val != "-" and after_season_val == "-":
            if age >= 30:
                return (float(add_correct_money(before_season.iloc[0]['Market Value']) * .8)), "Used Season Before * .8 But Had Both"
            else:
                return (float(add_correct_money(before_season.iloc[0]['Market Value']) * 1.2)), "Used Season Before * 1.2 But Had Both"
        elif before_season_val == "-" and after_season_val != "-":
            if age <= 30:
                return (float(add_correct_money(after_season.iloc[0]['Market Value']) * .8)), "Used Season After * .8 But Had Both"
            else:
                return (float(add_correct_money(after_season.iloc[0]['Market Value']) * 1.2)), "Used Season After * 1.2 But Had Both"
        else:
            return 0, "Had Both but Both Were Empty"


def find_in_transfermarkt(player_name, season, country_code, database_name):
    print(player_name, country_code, season)
    row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == season) & (database_name['Team 1 Code'] == country_code)]
    one_up_row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == int(season)+1) & (database_name['Team 1 Code'] == country_code)]
    one_down_row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == int(season)-1) & (database_name['Team 1 Code'] == country_code)]
    print(len(row), len(one_down_row), len(one_up_row))
    final_value = 0
    if not row.empty:
        #print('yeeea')
        player_id = row.iloc[0]['PlayerID']
        age = row.iloc[0]['Age']
        marketval = row.iloc[0]['Market Value']
        if type(marketval) == str:
            #print('der')
            if marketval == '-':
                marketval = 0
            elif '€' in marketval:
                marketval = add_correct_money(marketval)
            marketval = float((marketval)) #add_correct_money
        if marketval != 0:
            #print('wooo')
            #print('***', player_name, marketval)
            return marketval, "Found in Curr Season"
        else:
            #print(marketval)
            new_mv, reason = in_season_around(player_id, season, age)
            print(player_name, new_mv, reason)
            if new_mv == -1 or new_mv == 0 or new_mv == "-":
                return 0, reason
            else:
                return new_mv, reason
    elif not one_up_row.empty:
        player_id = one_up_row.iloc[0]['PlayerID']
        age = one_up_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    elif not one_down_row.empty:
        player_id = one_down_row.iloc[0]['PlayerID']
        age = one_down_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    else:
        return 0, "Did Not Find in Curr Season, One Up, or One Down"
    

### Euros qualifiers

In [8]:
lineups_euros_qualifiers = load_csv_dataset('Temp Lineup Data - Euros Qualifiers Lineups.csv')
#fix country names in the TeamName and Match 

# for index, row in lineups_euros_qualifiers.iterrows():
#     if row['TeamName'] == 'Czechia':
#         lineups_euros_qualifiers.at[index, 'TeamName'] = 'Czech'
#         lineups_euros_qualifiers.at[index, 'Match'] = lineups_euros_qualifiers.at[index, 'Match'].replace('Czechia', 'Czech')
#     if row['TeamName'] == 'Türkiye':
#         lineups_euros_qualifiers.at[index, 'TeamName'] = 'Turkey'
#         lineups_euros_qualifiers.at[index, 'Match'] = lineups_euros_qualifiers.at[index, 'Match'].replace('Türkiye', 'Turkey')
#     if row['TeamName'] == 'Bosnia':
#         lineups_euros_qualifiers.at[index, 'TeamName'] = 'Bosnia-Herzegovina'
#         lineups_euros_qualifiers.at[index, 'Match'] = lineups_euros_qualifiers.at[index, 'Match'].replace('Bosnia', 'Bosnia-Herzegovina')

#     if 'Czechia' in row['Match']:
#         lineups_euros_qualifiers.at[index, 'Match'] = lineups_euros_qualifiers.at[index, 'Match'].replace('Czechia', 'Czech')
#     if 'Türkiye' in row['Match']:
#         lineups_euros_qualifiers.at[index, 'Match'] = lineups_euros_qualifiers.at[index, 'Match'].replace('Türkiye', 'Turkey')
#     if 'Bosnia' in row['Match']:
#         if 'Bosnia-Herzegovina' in row['Match']:
#             0==0
#         else:
#             lineups_euros_qualifiers.at[index, 'Match'] = lineups_euros_qualifiers.at[index, 'Match'].replace('Bosnia', 'Bosnia-Herzegovina')

In [23]:
lineups_euros_qualifiers['Code'] = lineups_euros_qualifiers['TeamName'].apply(get_country_code)


In [ ]:
#Euros Qualifiers
#Macedonia --> North Macedonia
#Czechia --> Czech Republic
#N. Ireland --> Northern Ireland
for index, row in lineups_euros_qualifiers.iterrows():
    if row['TeamName'] == 'Macedonia':
        lineups_euros_qualifiers.at[index, 'TeamName'] = 'North Macedonia'
        lineups_euros_qualifiers.at[index, 'Match'] = lineups_euros_qualifiers.at[index, 'Match'].replace('Macedonia', 'North Macedonia')
    if row['TeamName'] == 'Czechia':
        lineups_euros_qualifiers.at[index, 'TeamName'] = 'Czech Republic'
        lineups_euros_qualifiers.at[index, 'Match'] = lineups_euros_qualifiers.at[index, 'Match'].replace('Czechia', 'Czech Republic')
    if row['TeamName'] == 'N. Ireland':
        lineups_euros_qualifiers.at[index, 'TeamName'] = 'Northern Ireland'
        lineups_euros_qualifiers.at[index, 'Match'] = lineups_euros_qualifiers.at[index, 'Match'].replace('N. Ireland', 'Northern Ireland')
lineups_euros_qualifiers['Code'] = lineups_euros_qualifiers['TeamName'].apply(get_country_code)


In [24]:
for index, row in lineups_euros_qualifiers.iterrows():
    country = row['TeamName']
    codes = countries_codes[countries_codes['Country'] == country][' Code'].unique()
    if len(codes) == 0:
        print(index, country)
    

In [25]:

for i in range(0, len(lineups_euros_qualifiers)):
    if lineups_euros_qualifiers.at[i, 'Date'].startswith('Sat '):
        lineups_euros_qualifiers.at[i, 'Date'] = lineups_euros_qualifiers.at[i, 'Date'].split('Sat ')[1]
    elif lineups_euros_qualifiers.at[i, 'Date'].startswith('Sun '):
        lineups_euros_qualifiers.at[i, 'Date'] = lineups_euros_qualifiers.at[i, 'Date'].split('Sun ')[1]
    elif lineups_euros_qualifiers.at[i, 'Date'].startswith('Wed '):
        lineups_euros_qualifiers.at[i, 'Date'] = lineups_euros_qualifiers.at[i, 'Date'].split('Wed ')[1]
    elif lineups_euros_qualifiers.at[i, 'Date'].startswith('Tue '):
        lineups_euros_qualifiers.at[i, 'Date'] = lineups_euros_qualifiers.at[i, 'Date'].split('Tue ')[1]
    elif lineups_euros_qualifiers.at[i, 'Date'].startswith('Thu '):
        lineups_euros_qualifiers.at[i, 'Date'] = lineups_euros_qualifiers.at[i, 'Date'].split('Thu ')[1]
    elif lineups_euros_qualifiers.at[i, 'Date'].startswith('Mon '):
        lineups_euros_qualifiers.at[i, 'Date'] = lineups_euros_qualifiers.at[i, 'Date'].split('Mon ')[1]
    elif lineups_euros_qualifiers.at[i, 'Date'].startswith('Fri '):
        lineups_euros_qualifiers.at[i, 'Date'] = lineups_euros_qualifiers.at[i, 'Date'].split('Fri ')[1]
lineups_euros_qualifiers['Date'] = pd.to_datetime(lineups_euros_qualifiers['Date'])

In [27]:
lineups_euros_qualifiers['Status New'] = ''

In [196]:
#old code to FIX the messed up match ID system
# for i in range(0, len(lineups_euros_qualifiers)):
#     index = lineups_euros_qualifiers.at[i, 'Unnamed: 0']
#     if index % 2 == 0:
#         correct_Index = round(index/2)
#         #print(correct_Index)
#     else:
#         correct_Index = round((index - 1)/2)
#         #print(correct_Index)
#     lineups_euros_qualifiers.at[i, 'Unnamed: 0'] = correct_Index



##OLD Match ID code


# import pandas as pd
# import datetime as datetime

# def convert_date(input_date):
#     formats = ["%d.%m.%y", "%d-%m-%Y", "%d-%b-%y", "%Y-%m-%d", "%b %d, %Y"]
#     target_format = "%Y-%m-%d"

#     for date_format in formats:
#         try:
#             # Try parsing the input date using the current format
#             parsed_date = datetime.strptime(input_date, date_format)
#             # If successful, convert it to the target format
#             result_date = parsed_date.strftime(target_format)
#             #year = parsed_date.year
#             #return year
#             return result_date
#         except ValueError:
#             #print(input_date)
#             pass


# #old_lineups_data['Formatted Date'] = old_lineups_data['Date'].apply(convert_date)
# lineups_data['Date'] = pd.to_datetime(lineups_data['Date'])

# dates_orig = lineups_euros_qualifiers['Date']
# lineups_euros_qualifiers['Date'] = pd.to_datetime(lineups_euros_qualifiers['Date'])

# lineups_euros_qualifiers['Match ID'] = ""
# counter = 0
# for index, row in lineups_euros_qualifiers.iterrows():
#     date = row['Date']
#     #date = convert_date(row['Date'])
#     date_range = pd.date_range(date - pd.Timedelta(days=1), date + pd.Timedelta(days=1))
#     matches = lineups_data[(lineups_data['Date'].isin(date_range)) & (lineups_data['Code'] == row['Code'])]
#     if not matches.empty:
#         lineups_euros_qualifiers.at[index, 'Match ID'] = matches.iloc[0]['Match ID']
#     else:
#         print(index, row['Match'])
#         counter += 1
# print(counter)
# lineups_euros_qualifiers['Date'] = dates_orig

0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36
37
37
38
38
39
39
40
40
41
41
42
42
43
43
44
44
45
45
46
46
47
47
48
48
49
49
50
50
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62
63
63
64
64
65
65
66
66
67
67
68
68
69
69
70
70
71
71
72
72
73
73
74
74
75
75
76
76
77
77
78
78
79
79
80
80
81
81
82
82
83
83
84
84
85
85
86
86
87
87
88
88
89
89
90
90
91
91
92
92
93
93
94
94
95
95
96
96
97
97
98
98
99
99
100
100
101
101
102
102
103
103
104
104
105
105
106
106
107
107
108
108
109
109
110
110
111
111
112
112
113
113
114
114
115
115
116
116
117
117
118
118
119
119
120
120
121
121
122
122
123
123
124
124
125
125
126
126
127
127
128
128
129
129
130
130
131
131
132
132
133
133
134
134
135
135
136
136
137
137
138
138
139
139
140
140
141
141
142
142
143
143
144
144
145
145
146
146
147
147
148
148
149
149
150
150
151
151
152


In [219]:
lineups_euros_qualifiers = lineups_euros_qualifiers.drop(columns={'Match ID'})
lineups_euros_qualifiers['Match ID'] = ''
for index, row in lineups_euros_qualifiers.iterrows():
    comp = row['Competition General']
    index_of_unnamed = lineups_euros_qualifiers.at[index, 'Unnamed: 0']
    match_ID = f"{comp}_{index_of_unnamed}"
    lineups_euros_qualifiers.at[index, 'Match ID'] = match_ID

In [221]:
lookup_dataset_1 = pd.DataFrame()


not_found_dataset = pd.DataFrame()


In [222]:
counter = 0
salary_boolean = False
#Start from index 2
for k in range(0, len(lineups_euros_qualifiers)): #len(lineups_euros_qualifiers)

    nationality_this = lineups_euros_qualifiers.at[k, 'TeamName']
    competition_this = lineups_euros_qualifiers.at[k, 'Competition']
    #nationality_this = lineups_data.at[k, 'Match']
    date_this = lineups_euros_qualifiers.at[k, 'Date']
    country_code_this = lineups_euros_qualifiers.at[k, 'Code']
    date_this = str(date_this)
    if ' 00:00:00' in date_this:
        date_this = date_this.split(' 00:00:00')[0]
    print(k, nationality_this, date_this)
    for i in range(2, 13): 
        number = -999
        reason = ''

        column = lineups_euros_qualifiers.columns[i]
        player_this = remove_prefix(lineups_euros_qualifiers.at[k, f'{column}'])
        if player_this[0] == ' ':
            player_this = player_this[1:]

        if player_this == '-':
            0==0
        else:
            #print(player_this)
            
            if '-' in date_this:
                if len(date_this.split("-")[2]) == 4:
                    yearstr = date_this.split("-")[2] #this is a 4 digit number
                    full_num = int(yearstr)
                elif len(date_this.split("-")[2]) == 2:
                    if len(date_this.split("-")[0]) == 4:
                        yearstr = date_this.split("-")[0]
                        full_num = int(yearstr)
                    else:
                        yearstr = date_this.split("-")[2] #this is a 2 digit number
                        full_num = int('20' + yearstr)
        dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == country_code_this) & 
                                        (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
        if player_this in dataset_nationality:
            0==0
            
            number, reason = find_in_transfermarkt(player_this, full_num, country_code_this, leagues_value)
            #print(player_this, reason, add_correct_money(number))

            #add a new row to lookup dataset 1 
            if type(number) == float:
                0==0
            else:
                number = add_correct_money(number)
            if i == 13:
                lineups_euros_qualifiers.at[k, 'Status New'] = 'Done'

            data_row = {
                'Match ID': lineups_euros_qualifiers.at[k, 'Match ID'],
                'Competition': competition_this,
                'Match': lineups_euros_qualifiers.at[k, 'Match'],
                'Date': date_this,
                'Nationality': nationality_this,
                'Team Country Code': country_code_this,
                'Season': full_num,
                'Status': 'SUCCESS',
                'Name': player_this, #this will be zero if a fail happens 
                'Market Value': number,
                'Lookup Still Required?': False,
                'Lookup Return Case': reason,
                'Impute Required?': False,
                'Name(s) Found': player_this,
                'ORIGINAL JERSEY': lineups_euros_qualifiers.at[k, f'{column}'],
                'Match Case': 'single' #single, multiple, none or cyrillic
            }
            lookup_dataset_1 = pd.concat([lookup_dataset_1, pd.DataFrame([data_row])], ignore_index=True)
            
        else:
            print(player_this, 'nope')
            # counter += 1
            dataset_nationality_all_years = leagues_value[(leagues_value['Team 1 Code'] == country_code_this)]['Name'].unique()
            if player_this in dataset_nationality_all_years:
                
                print('***Found Him', player_this)
                data_row = {
                    'Match ID': lineups_euros_qualifiers.at[k, 'Match ID'],
                    'Competition': competition_this,
                    'Match': lineups_euros_qualifiers.at[k, 'Match'],
                    'Date': date_this,
                    'Nationality': nationality_this,
                    'Team Country Code': country_code_this,
                    'Season': full_num,
                    'Status': 'FAIL',
                    'Name': player_this, #this will be zero if a fail happens 
                    'Market Value': 0,
                    'Lookup Still Required?': True,
                    'Lookup Return Case': 'Did Not Find in Curr Season, One Up, or One Down', 
                    'Impute Required?': False,
                    'Name(s) Found': player_this,
                    'ORIGINAL JERSEY': lineups_euros_qualifiers.at[k, f'{column}'],
                    'Match Case': 'single' #single, multiple, none or cyrillic
                }        
                not_found_dataset = pd.concat([not_found_dataset, pd.DataFrame([data_row])], ignore_index=True)    
            else:
                print('DID NOT FIND ', player_this)
                counter += 1
                data_row = {
                    'Match ID': lineups_euros_qualifiers.at[k, 'Match ID'],
                    'Competition': competition_this,
                    'Match': lineups_euros_qualifiers.at[k, 'Match'],
                    'Date': date_this,
                    'Nationality': nationality_this,
                    'Team Country Code': country_code_this,
                    'Season': full_num,
                    'Status': 'FAIL',
                    'Name': player_this, #this will be zero if a fail happens 
                    'Market Value': 0,
                    'Lookup Still Required?': True,
                    'Lookup Return Case': 'Did Not Find in any Season', #Did Not Find in Curr Season, One Up, or One Down
                    'Impute Required?': False,
                    'Name(s) Found': player_this,
                    'ORIGINAL JERSEY': lineups_euros_qualifiers.at[k, f'{column}'],
                    'Match Case': 'single' #single, multiple, none or cyrillic
                }        
                not_found_dataset = pd.concat([not_found_dataset, pd.DataFrame([data_row])], ignore_index=True)     
            
    #dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == country_code_this]['Name'].unique()
        #make the dataset nationality equal to only the season before , current or after 

print(counter)



0 Kazakhstan 2014-09-09
Andrey Sidelnikov KZ 2014
1 1 0
Ilya Vorotnikov KZ 2014
1 0 0
Renat Abdulin KZ 2014
1 1 0
Yuriy Logvinenko KZ 2014
1 1 1
Samat Smakov KZ 2014
1 0 1
Anatoliy Bogdanov KZ 2014
1 1 0
Dmitriy Miroshnichenko KZ 2014
1 0 0
Dmitriy Shomko KZ 2014
1 1 1
Bauyrzhan Islamkhan KZ 2014
1 1 1
Tanat Nuserbaev KZ 2014
1 1 1
Sergey Khizhnichenko KZ 2014
1 1 1
1 Latvia 2014-09-09
Aleksandrs Kolinko LV 2014
1 1 0
Kaspars Gorkss LV 2014
1 1 1
Nauris Bulvitis LV 2014
1 1 0
Vitalijs Maksimenko LV 2014
1 1 1
Vladislavs Gabovs LV 2014
1 1 1
Artis Lazdins LV 2014
1 1 1
Arturs Zjuzins LV 2014
1 1 1
Andrejs Kovalovs LV 2014
1 1 1
Olegs Laizans LV 2014
1 1 1
Valerijs Sabala LV 2014
1 1 1
Artjoms Rudnevs LV 2014
1 1 1
2 Czech Republic 2014-09-09
Petr Cech CZ 2014
1 1 1
Michal Kadlec CZ 2014
1 1 1
Vaclav Prochazka CZ 2014
1 1 1
David Limbersky CZ 2014
1 1 1
Pavel Kaderabek CZ 2014
1 1 1
Lukas Vacha CZ 2014
1 1 0
Vladimír Darida nope
***Found Him Vladimír Darida
Borek Dockal CZ 2014
1 1 1
Lad

In [225]:
#not_found_dataset = not_found_dataset_copy# = not_found_dataset
not_found_dataset['Status New'] = ''
#not_found_dataset_copy = not_found_dataset

In [122]:
# for index,row in not_found_dataset.iterrows():
#     if row['Status New'] != 'Done':
#         player_this = remove_prefix(row['ORIGINAL JERSEY'])
#         print(player_this)
#         if player_this[0] == ' ':
#             print('ya')
#             player_this = player_this[1:]
#             print('now')
#             print(player_this, player_this in leagues_value['Name'].unique(), unidecode(player_this) in leagues_value['Name'].unique())
#         #print(unidecode(player_this) in leagues_value['Name'].unique())



 Ucha Lobjanidze
ya
now
Ucha Lobjanidze True True
 Ucha Lobjanidze
ya
now
Ucha Lobjanidze True True
 Ucha Lobjanidze
ya
now
Ucha Lobjanidze True True
 Ucha Lobjanidze
ya
now
Ucha Lobjanidze True True
 Ucha Lobjanidze
ya
now
Ucha Lobjanidze True True
 Ucha Lobjanidze
ya
now
Ucha Lobjanidze True True
 Ucha Lobjanidze
ya
now
Ucha Lobjanidze True True
 Ucha Lobjanidze
ya
now
Ucha Lobjanidze True True
 Ucha Lobjanidze
ya
now
Ucha Lobjanidze True True
 Gökhan Inler
ya
now
Gökhan Inler True False
 Jasmin Kurtic
ya
now
Jasmin Kurtic True True
 Gökhan Inler
ya
now
Gökhan Inler True False
 Jasmin Kurtic
ya
now
Jasmin Kurtic True True
 Jasmin Kurtic
ya
now
Jasmin Kurtic True True
 Gökhan Inler
ya
now
Gökhan Inler True False
 Jasmin Kurtic
ya
now
Jasmin Kurtic True True
 Gökhan Inler
ya
now
Gökhan Inler True False
 Jasmin Kurtic
ya
now
Jasmin Kurtic True True
 Gökhan Inler
ya
now
Gökhan Inler True False
 Jasmin Kurtic
ya
now
Jasmin Kurtic True True
 Jasmin Kurtic
ya
now
Jasmin Kurtic True True
 Gö

In [226]:
for index,row in not_found_dataset.iterrows():
    if row['Status New'] != 'Done':
        #time.sleep(1)
        player_this = remove_prefix(row['ORIGINAL JERSEY'])
        if player_this[0] == ' ':
                player_this = player_this[1:]
        code = row['Team Country Code']
        full_num = row['Season']
        dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == code) & 
                                            (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
        print(player_this)
        if unidecode(player_this) in dataset_nationality:
            print(index, player_this, 'ya')
            number, reason = find_in_transfermarkt(unidecode(player_this), full_num, code, leagues_value)
            print(number, reason)
            if type(number) == float:
                0==0
            else:
                number = add_correct_money(number)

                
            
            not_found_dataset.at[index, 'Market Value'] = number
            not_found_dataset.at[index, 'Lookup Return Case'] = reason
            not_found_dataset.at[index, 'Lookup Still Required?'] = False
            not_found_dataset.at[index, 'Status'] = 'SUCCESS'
            not_found_dataset.at[index, 'Status New'] = 'Done'

            # data_row = {
            #     'Match ID': row['Match ID'],
            #     'Competition': row['Competition'],
            #     'Match': row['Match'],
            #     'Date': row['Date'],
            #     'Nationality': row['Nationality'],
            #     'Team Country Code': row['Team Country Code'],
            #     'Season': row['Season'],
            #     'Status': 'SUCCESS',
            #     'Name': player_this, #this will be zero if a fail happens 
            #     'Market Value': number,
            #     'Lookup Still Required?': False,
            #     'Lookup Return Case': reason,
            #     'Impute Required?': False,
            #     'Name(s) Found': player_this,
            #     'ORIGINAL JERSEY': lineups_euros_qualifiers.at[k, f'{column}'],
            #     'Match Case': 'single' #single, multiple, none or cyrillic
            # }
            # lookup_dataset_1 = pd.concat([lookup_dataset_1, pd.DataFrame([data_row])], ignore_index=True)
        else:
            print(index, player_this, 'nein')
            if player_this in leagues_value[leagues_value['Team 1 Code'] == code]['Name'].unique():
                # not_found_dataset.at[index, 'Market Value'] = number
                not_found_dataset.at[index, 'Lookup Return Case'] = reason
                not_found_dataset.at[index, 'Lookup Still Required?'] = True
                not_found_dataset.at[index, 'Status'] = 'FAIL - NEW'
                not_found_dataset.at[index, 'Status New'] = 'Done'
                print('finished ', player_this)
    else:
        0==0
        #print('Done Case')


Vladimír Darida
0 Vladimír Darida ya
Vladimir Darida CZ 2014
1 1 1
7000000.0 Found in Curr Season
Vladimír Darida
1 Vladimír Darida ya
Vladimir Darida CZ 2014
1 1 1
7000000.0 Found in Curr Season
Vladimír Darida
2 Vladimír Darida ya
Vladimir Darida CZ 2014
1 1 1
7000000.0 Found in Curr Season
Vladimír Darida
3 Vladimír Darida ya
Vladimir Darida CZ 2014
1 1 1
7000000.0 Found in Curr Season
Vladimír Darida
4 Vladimír Darida ya
Vladimir Darida CZ 2015
1 1 1
6000000.0 Found in Curr Season
Vladimír Darida
5 Vladimír Darida ya
Vladimir Darida CZ 2015
1 1 1
6000000.0 Found in Curr Season
Vladimír Darida
6 Vladimír Darida ya
Vladimir Darida CZ 2015
1 1 1
6000000.0 Found in Curr Season
Vladimír Darida
7 Vladimír Darida ya
Vladimir Darida CZ 2015
1 1 1
6000000.0 Found in Curr Season
Vladimír Darida
8 Vladimír Darida ya
Vladimir Darida CZ 2015
1 1 1
6000000.0 Found in Curr Season
Norbert Gyömbér
9 Norbert Gyömbér ya
Norbert Gyomber SK 2014
1 1 1
800000.0 Found in Curr Season
Peter Pekarík
10 Pete

In [227]:
not_found_dataset[not_found_dataset['Status New'] == '']['Name'].value_counts()

Name
Dennis Salanovic    23
Dorian Railean       8
Horațiu Moldovan     8
Marko Dmitrovic      7
Ionuț Nedelcearu     7
Emerson              6
Olimpiu Moruțan      4
Ismail Isa           3
Deni Hocko           2
Denis Drăguș         2
Ísak Jóhannesson     2
Rashad Eyyubov       1
Name: count, dtype: int64

In [228]:
not_found_dataset[not_found_dataset['Status'] == 'FAIL - NEW']['Name'].value_counts()

Name
Jasmin Kurtic      19
Odmar Færø         17
Ucha Lobjanidze     9
Gökhan Inler        8
Name: count, dtype: int64

In [229]:
#things to fix 

#change the gorup thatis fail new to manual somehow 


#the match ID system is wrong 

for index, row in not_found_dataset.iterrows():
    if row['Status New'] == '':
        not_found_dataset.at[index, 'Status'] = 'FAIL - MANUAL'



In [230]:
final_dataset_euros_qualifiers = pd.concat([lookup_dataset_1, not_found_dataset], ignore_index=True)

In [236]:
final_dataset_euros_qualifiers.to_csv('euro_qualis_lineups_data_complete.csv')

In [234]:
# Grouping by 'Match', 'Date', and 'Match ID' columns and counting rows
grouped_df = final_dataset_euros_qualifiers.groupby(['Match ID']).size().reset_index(name='Count')

# # Extracting unique Match IDs for each unique combination of 'Match' and 'Date'
# unique_match_ids = final_dataset_euros_qualifiers.groupby(['Match', 'Date'])['Match ID'].unique().reset_index(name='Unique Match IDs')

# # Merging the two results on 'Match' and 'Date' columns
# result = pd.merge(grouped_df, unique_match_ids, on=['Match', 'Date'])
# result[result['Count'] == 11]
grouped_df

,Match ID,Count
0,"Euros, Qualifiers_0",22
1,"Euros, Qualifiers_1",22
2,"Euros, Qualifiers_10",22
3,"Euros, Qualifiers_100",22
4,"Euros, Qualifiers_101",22
...,...,...
735,"Euros, Qualifiers_95",22
736,"Euros, Qualifiers_96",22
737,"Euros, Qualifiers_97",22
738,"Euros, Qualifiers_98",22


## Euros 

In [264]:
lineups_euros = load_csv_dataset('NEW LINEUPS to look up/Temp Lineup Data - Euros Lineups.csv')

In [265]:
lineups_euros['Code'] = lineups_euros['TeamName'].apply(get_country_code)


In [266]:
for index, row in lineups_euros.iterrows():
    if row['TeamName'] in countries_codes['Country'].unique():
        0==0
    else:
        print(row['TeamName'])

Czechia
Czechia
Czechia
Czechia
Czechia
N. Ireland
N. Ireland
N. Ireland
N. Ireland
Czechia
Czechia
Czechia


In [267]:
#Euros Qualifiers
#Macedonia --> North Macedonia
#Czechia --> Czech Republic
#N. Ireland --> Northern Ireland
for index, row in lineups_euros.iterrows():
    if row['TeamName'] == 'Macedonia':
        lineups_euros.at[index, 'TeamName'] = 'North Macedonia'
        lineups_euros.at[index, 'Match'] = lineups_euros.at[index, 'Match'].replace('Macedonia', 'North Macedonia')
    if row['TeamName'] == 'Czechia':
        lineups_euros.at[index, 'TeamName'] = 'Czech Republic'
        lineups_euros.at[index, 'Match'] = lineups_euros.at[index, 'Match'].replace('Czechia', 'Czech Republic')
    if row['TeamName'] == 'N. Ireland':
        lineups_euros.at[index, 'TeamName'] = 'Northern Ireland'
        lineups_euros.at[index, 'Match'] = lineups_euros.at[index, 'Match'].replace('N. Ireland', 'Northern Ireland')
lineups_euros['Code'] = lineups_euros['TeamName'].apply(get_country_code)


In [268]:

for i in range(0, len(lineups_euros)):
    if lineups_euros.at[i, 'Date'].startswith('Sat '):
        lineups_euros.at[i, 'Date'] = lineups_euros.at[i, 'Date'].split('Sat ')[1]
    elif lineups_euros.at[i, 'Date'].startswith('Sun '):
        lineups_euros.at[i, 'Date'] = lineups_euros.at[i, 'Date'].split('Sun ')[1]
    elif lineups_euros.at[i, 'Date'].startswith('Wed '):
        lineups_euros.at[i, 'Date'] = lineups_euros.at[i, 'Date'].split('Wed ')[1]
    elif lineups_euros.at[i, 'Date'].startswith('Tue '):
        lineups_euros.at[i, 'Date'] = lineups_euros.at[i, 'Date'].split('Tue ')[1]
    elif lineups_euros.at[i, 'Date'].startswith('Thu '):
        lineups_euros.at[i, 'Date'] = lineups_euros.at[i, 'Date'].split('Thu ')[1]
    elif lineups_euros.at[i, 'Date'].startswith('Mon '):
        lineups_euros.at[i, 'Date'] = lineups_euros.at[i, 'Date'].split('Mon ')[1]
    elif lineups_euros.at[i, 'Date'].startswith('Fri '):
        lineups_euros.at[i, 'Date'] = lineups_euros.at[i, 'Date'].split('Fri ')[1]
lineups_euros['Date'] = pd.to_datetime(lineups_euros['Date'])

lineups_euros['Status New'] = ''

#lineups_euros = lineups_euros.drop(columns={'Match ID'})
lineups_euros['Match ID'] = ''
for index, row in lineups_euros.iterrows():
    comp = row['Competition General']
    index_of_unnamed = lineups_euros.at[index, 'Unnamed: 0']
    match_ID = f"{comp}_{index_of_unnamed}"
    lineups_euros.at[index, 'Match ID'] = match_ID

lookup_dataset_1 = pd.DataFrame()


not_found_dataset = pd.DataFrame()


In [270]:
counter = 0
salary_boolean = False
#Start from index 2
for k in range(0, len(lineups_euros)): #len(lineups_euros_qualifiers)

    nationality_this = lineups_euros.at[k, 'TeamName']
    competition_this = lineups_euros.at[k, 'Competition']
    #nationality_this = lineups_data.at[k, 'Match']
    date_this = lineups_euros.at[k, 'Date']
    country_code_this = lineups_euros.at[k, 'Code']
    date_this = str(date_this)
    if ' 00:00:00' in date_this:
        date_this = date_this.split(' 00:00:00')[0]
    print(k, nationality_this, date_this)
    for i in range(2, 13): 
        number = -999
        reason = ''

        column = lineups_euros.columns[i]
        player_this = remove_prefix(lineups_euros.at[k, f'{column}'])
        if player_this[0] == ' ':
            player_this = player_this[1:]

        if player_this == '-':
            0==0
        else:
            #print(player_this)
            
            if '-' in date_this:
                if len(date_this.split("-")[2]) == 4:
                    yearstr = date_this.split("-")[2] #this is a 4 digit number
                    full_num = int(yearstr)
                elif len(date_this.split("-")[2]) == 2:
                    if len(date_this.split("-")[0]) == 4:
                        yearstr = date_this.split("-")[0]
                        full_num = int(yearstr)
                    else:
                        yearstr = date_this.split("-")[2] #this is a 2 digit number
                        full_num = int('20' + yearstr)
        dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == country_code_this) & 
                                        (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
        if player_this in dataset_nationality:
            0==0
            
            number, reason = find_in_transfermarkt(player_this, full_num, country_code_this, leagues_value)
            #print(player_this, reason, add_correct_money(number))

            #add a new row to lookup dataset 1 
            if type(number) == float:
                0==0
            else:
                number = add_correct_money(number)
            if i == 13:
                lineups_euros.at[k, 'Status New'] = 'Done'

            data_row = {
                'Match ID': lineups_euros.at[k, 'Match ID'],
                'Competition': competition_this,
                'Match': lineups_euros.at[k, 'Match'],
                'Date': date_this,
                'Nationality': nationality_this,
                'Team Country Code': country_code_this,
                'Season': full_num,
                'Status': 'SUCCESS',
                'Name': player_this, #this will be zero if a fail happens 
                'Market Value': number,
                'Lookup Still Required?': False,
                'Lookup Return Case': reason,
                'Impute Required?': False,
                'Name(s) Found': player_this,
                'ORIGINAL JERSEY': lineups_euros.at[k, f'{column}'],
                'Match Case': 'single' #single, multiple, none or cyrillic
            }
            lookup_dataset_1 = pd.concat([lookup_dataset_1, pd.DataFrame([data_row])], ignore_index=True)
            
        else:
            print(player_this, 'nope')
            # counter += 1
            dataset_nationality_all_years = leagues_value[(leagues_value['Team 1 Code'] == country_code_this)]['Name'].unique()
            if player_this in dataset_nationality_all_years:
                
                print('***Found Him', player_this)
                data_row = {
                    'Match ID': lineups_euros.at[k, 'Match ID'],
                    'Competition': competition_this,
                    'Match': lineups_euros.at[k, 'Match'],
                    'Date': date_this,
                    'Nationality': nationality_this,
                    'Team Country Code': country_code_this,
                    'Season': full_num,
                    'Status': 'FAIL',
                    'Name': player_this, #this will be zero if a fail happens 
                    'Market Value': 0,
                    'Lookup Still Required?': True,
                    'Lookup Return Case': 'Did Not Find in Curr Season, One Up, or One Down', 
                    'Impute Required?': False,
                    'Name(s) Found': player_this,
                    'ORIGINAL JERSEY': lineups_euros.at[k, f'{column}'],
                    'Match Case': 'single' #single, multiple, none or cyrillic
                }        
                not_found_dataset = pd.concat([not_found_dataset, pd.DataFrame([data_row])], ignore_index=True)    
            else:
                print('DID NOT FIND ', player_this)
                counter += 1
                data_row = {
                    'Match ID': lineups_euros.at[k, 'Match ID'],
                    'Competition': competition_this,
                    'Match': lineups_euros.at[k, 'Match'],
                    'Date': date_this,
                    'Nationality': nationality_this,
                    'Team Country Code': country_code_this,
                    'Season': full_num,
                    'Status': 'FAIL',
                    'Name': player_this, #this will be zero if a fail happens 
                    'Market Value': 0,
                    'Lookup Still Required?': True,
                    'Lookup Return Case': 'Did Not Find in any Season', #Did Not Find in Curr Season, One Up, or One Down
                    'Impute Required?': False,
                    'Name(s) Found': player_this,
                    'ORIGINAL JERSEY': lineups_euros.at[k, f'{column}'],
                    'Match Case': 'single' #single, multiple, none or cyrillic
                }        
                not_found_dataset = pd.concat([not_found_dataset, pd.DataFrame([data_row])], ignore_index=True)     
            
    #dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == country_code_this]['Name'].unique()
        #make the dataset nationality equal to only the season before , current or after 

print(counter)



0 Italy 2021-07-11
Gianluigi Donnarumma IT 2021
1 1 1
Giorgio Chiellini IT 2021
1 1 0
Leonardo Bonucci IT 2021
1 1 1
Emerson nope
DID NOT FIND  Emerson
Giovanni Di Lorenzo IT 2021
1 1 1
Jorginho IT 2021
1 1 1
Marco Verratti IT 2021
1 1 1
Nicolò Barella IT 2021
1 1 1
Lorenzo Insigne IT 2021
1 1 0
Federico Chiesa IT 2021
0 1 1
Ciro Immobile IT 2021
1 1 1
1 England 2021-07-11
Jordan Pickford EN 2021
1 1 1
John Stones EN 2021
1 1 1
Harry Maguire EN 2021
1 1 1
Kyle Walker EN 2021
1 1 1
Declan Rice EN 2021
1 1 1
Kalvin Phillips EN 2021
1 1 1
Kieran Trippier EN 2021
1 1 1
Luke Shaw EN 2021
1 1 1
Mason Mount EN 2021
1 1 1
Raheem Sterling EN 2021
1 1 1
Harry Kane EN 2021
1 1 1
2 Italy 2021-07-06
Gianluigi Donnarumma IT 2021
1 1 1
Giorgio Chiellini IT 2021
1 1 0
Leonardo Bonucci IT 2021
1 1 1
Emerson nope
DID NOT FIND  Emerson
Giovanni Di Lorenzo IT 2021
1 1 1
Jorginho IT 2021
1 1 1
Marco Verratti IT 2021
1 1 1
Nicolò Barella IT 2021
1 1 1
Lorenzo Insigne IT 2021
1 1 0
Federico Chiesa IT 2021
0 

In [285]:
not_found_dataset['Status New'] = ''

In [288]:
for index,row in not_found_dataset.iterrows():
    if row['Status New'] != 'Done':
        #time.sleep(1)
        player_this = remove_prefix(row['ORIGINAL JERSEY'])
        if player_this[0] == ' ':
                player_this = player_this[1:]
        code = row['Team Country Code']
        full_num = row['Season']
        dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == code) & 
                                            (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
        print(player_this)
        if unidecode(player_this) in dataset_nationality:
            print(index, player_this, 'ya')
            number, reason = find_in_transfermarkt(unidecode(player_this), full_num, code, leagues_value)
            print(number, reason)
            if type(number) == float:
                0==0
            else:
                number = add_correct_money(number)  
            
            not_found_dataset.at[index, 'Market Value'] = number
            not_found_dataset.at[index, 'Lookup Return Case'] = reason
            not_found_dataset.at[index, 'Lookup Still Required?'] = False
            not_found_dataset.at[index, 'Status'] = 'SUCCESS'
            not_found_dataset.at[index, 'Status New'] = 'Done'

            # data_row = {
            #     'Match ID': row['Match ID'],
            #     'Competition': row['Competition'],
            #     'Match': row['Match'],
            #     'Date': row['Date'],
            #     'Nationality': row['Nationality'],
            #     'Team Country Code': row['Team Country Code'],
            #     'Season': row['Season'],
            #     'Status': 'SUCCESS',
            #     'Name': player_this, #this will be zero if a fail happens 
            #     'Market Value': number,
            #     'Lookup Still Required?': False,
            #     'Lookup Return Case': reason,
            #     'Impute Required?': False,
            #     'Name(s) Found': player_this,
            #     'ORIGINAL JERSEY': lineups_euros_qualifiers.at[k, f'{column}'],
            #     'Match Case': 'single' #single, multiple, none or cyrillic
            # }
            # lookup_dataset_1 = pd.concat([lookup_dataset_1, pd.DataFrame([data_row])], ignore_index=True)
        else:
            print(index, player_this, 'nein')
            if player_this in leagues_value[leagues_value['Team 1 Code'] == code]['Name'].unique():
                # not_found_dataset.at[index, 'Market Value'] = number
                not_found_dataset.at[index, 'Lookup Return Case'] = reason
                not_found_dataset.at[index, 'Lookup Still Required?'] = True
                not_found_dataset.at[index, 'Status'] = 'FAIL - NEW'
                not_found_dataset.at[index, 'Status New'] = 'Done'
                print('finished ', player_this)
    else:
        0==0
        #print('Done Case')

Emerson
0 Emerson nein
Emerson
1 Emerson nein
Emerson
4 Emerson nein


In [290]:
for index, row in not_found_dataset.iterrows():
    if row['Status New'] == '':
        not_found_dataset.at[index, 'Status'] = 'FAIL - NEW'

In [292]:
final_dataset_euros = pd.concat([lookup_dataset_1, not_found_dataset], ignore_index=True)

In [293]:
final_dataset_euros

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Status New
0,Euros_0,Euros 2020,Italy vs England,2021-07-11,Italy,IT,2021,SUCCESS,Gianluigi Donnarumma,60000000.0,False,Found in Curr Season,False,Gianluigi Donnarumma,21 Gianluigi Donnarumma,single,NaN
1,Euros_0,Euros 2020,Italy vs England,2021-07-11,Italy,IT,2021,SUCCESS,Giorgio Chiellini,2000000.0,False,Found in Curr Season,False,Giorgio Chiellini,3 Giorgio Chiellini,single,NaN
2,Euros_0,Euros 2020,Italy vs England,2021-07-11,Italy,IT,2021,SUCCESS,Leonardo Bonucci,13000000.0,False,Found in Curr Season,False,Leonardo Bonucci,19 Leonardo Bonucci,single,NaN
3,Euros_0,Euros 2020,Italy vs England,2021-07-11,Italy,IT,2021,SUCCESS,Giovanni Di Lorenzo,22000000.0,False,Found in Curr Season,False,Giovanni Di Lorenzo,2 Giovanni Di Lorenzo,single,NaN
4,Euros_0,Euros 2020,Italy vs England,2021-07-11,Italy,IT,2021,SUCCESS,Jorginho,50000000.0,False,Found in Curr Season,False,Jorginho,8 Jorginho,single,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2239,Euros_84,Euros 2016,Türkiye vs Croatia,2016-06-12,Croatia,HR,2016,SUCCESS,Mario Mandžukić,20000000.0,False,Found in Curr Season,False,Mario Mandžukić,17 Mario Mandžukić,single,Done
2240,Euros_85,Euros 2016,Spain vs Czech Republic,2016-06-13,Czech Republic,CZ,2016,SUCCESS,Vladimír Darida,10000000.0,False,Found in Curr Season,False,Vladimír Darida,22 Vladimír Darida,single,Done
2241,Euros_86,Euros 2016,Czech Republic vs Croatia,2016-06-17,Czech Republic,CZ,2016,SUCCESS,Vladimír Darida,10000000.0,False,Found in Curr Season,False,Vladimír Darida,22 Vladimír Darida,single,Done
2242,Euros_86,Euros 2016,Czechia vs Croatia,2016-06-17,Croatia,HR,2016,SUCCESS,Mario Mandžukić,20000000.0,False,Found in Curr Season,False,Mario Mandžukić,17 Mario Mandžukić,single,Done


In [294]:
final_dataset_euros.to_csv('euros_lineups_data_complete.csv')

### World Cup

In [307]:
world_cup_lineups = load_csv_dataset('NEW LINEUPS to look up/Temp Lineup Data - World Cup Lineups.csv')

In [308]:
dataset_name = world_cup_lineups

In [309]:
dataset_name['Code'] = dataset_name['TeamName'].apply(get_country_code)


In [310]:
dataset_name['Code'] = dataset_name['TeamName'].apply(get_country_code)

for i in range(0, len(dataset_name)):
    if dataset_name.at[i, 'Date'].startswith('Sat '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Sat ')[1]
    elif dataset_name.at[i, 'Date'].startswith('Sun '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Sun ')[1]
    elif dataset_name.at[i, 'Date'].startswith('Wed '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Wed ')[1]
    elif dataset_name.at[i, 'Date'].startswith('Tue '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Tue ')[1]
    elif dataset_name.at[i, 'Date'].startswith('Thu '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Thu ')[1]
    elif dataset_name.at[i, 'Date'].startswith('Mon '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Mon ')[1]
    elif dataset_name.at[i, 'Date'].startswith('Fri '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Fri ')[1]
dataset_name['Date'] = pd.to_datetime(dataset_name['Date'])

dataset_name['Status New'] = ''

#lineups_euros = lineups_euros.drop(columns={'Match ID'})
dataset_name['Match ID'] = ''
for index, row in dataset_name.iterrows():
    comp = row['Competition General']
    index_of_unnamed = dataset_name.at[index, 'Unnamed: 0']
    match_ID = f"{comp}_{index_of_unnamed}"
    dataset_name.at[index, 'Match ID'] = match_ID

lookup_dataset_1 = pd.DataFrame()


not_found_dataset = pd.DataFrame()


In [311]:
counter = 0
salary_boolean = False
#Start from index 2
for k in range(0, len(dataset_name)): #len(lineups_euros_qualifiers)

    nationality_this = dataset_name.at[k, 'TeamName']
    competition_this = dataset_name.at[k, 'Competition']
    #nationality_this = lineups_data.at[k, 'Match']
    date_this = dataset_name.at[k, 'Date']
    country_code_this = dataset_name.at[k, 'Code']
    date_this = str(date_this)
    if ' 00:00:00' in date_this:
        date_this = date_this.split(' 00:00:00')[0]
    print(k, nationality_this, date_this)
    for i in range(2, 13): 
        number = -999
        reason = ''

        column = dataset_name.columns[i]
        player_this = remove_prefix(dataset_name.at[k, f'{column}'])
        if player_this[0] == ' ':
            player_this = player_this[1:]

        if player_this == '-':
            0==0
        else:
            #print(player_this)
            
            if '-' in date_this:
                if len(date_this.split("-")[2]) == 4:
                    yearstr = date_this.split("-")[2] #this is a 4 digit number
                    full_num = int(yearstr)
                elif len(date_this.split("-")[2]) == 2:
                    if len(date_this.split("-")[0]) == 4:
                        yearstr = date_this.split("-")[0]
                        full_num = int(yearstr)
                    else:
                        yearstr = date_this.split("-")[2] #this is a 2 digit number
                        full_num = int('20' + yearstr)
        dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == country_code_this) & 
                                        (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
        if player_this in dataset_nationality:
            0==0
            
            number, reason = find_in_transfermarkt(player_this, full_num, country_code_this, leagues_value)
            #print(player_this, reason, add_correct_money(number))

            #add a new row to lookup dataset 1 
            if type(number) == float:
                0==0
            else:
                number = add_correct_money(number)
            if i == 13:
                lineups_euros.at[k, 'Status New'] = 'Done'

            data_row = {
                'Match ID': dataset_name.at[k, 'Match ID'],
                'Competition': competition_this,
                'Match': dataset_name.at[k, 'Match'],
                'Date': date_this,
                'Nationality': nationality_this,
                'Team Country Code': country_code_this,
                'Season': full_num,
                'Status': 'SUCCESS',
                'Name': player_this, #this will be zero if a fail happens 
                'Market Value': number,
                'Lookup Still Required?': False,
                'Lookup Return Case': reason,
                'Impute Required?': False,
                'Name(s) Found': player_this,
                'ORIGINAL JERSEY': dataset_name.at[k, f'{column}'],
                'Match Case': 'single' #single, multiple, none or cyrillic
            }
            lookup_dataset_1 = pd.concat([lookup_dataset_1, pd.DataFrame([data_row])], ignore_index=True)
            
        else:
            print(player_this, 'nope')
            # counter += 1
            dataset_nationality_all_years = leagues_value[(leagues_value['Team 1 Code'] == country_code_this)]['Name'].unique()
            if player_this in dataset_nationality_all_years:
                
                print('***Found Him', player_this)
                data_row = {
                    'Match ID': dataset_name.at[k, 'Match ID'],
                    'Competition': competition_this,
                    'Match': dataset_name.at[k, 'Match'],
                    'Date': date_this,
                    'Nationality': nationality_this,
                    'Team Country Code': country_code_this,
                    'Season': full_num,
                    'Status': 'FAIL',
                    'Name': player_this, #this will be zero if a fail happens 
                    'Market Value': 0,
                    'Lookup Still Required?': True,
                    'Lookup Return Case': 'Did Not Find in Curr Season, One Up, or One Down', 
                    'Impute Required?': False,
                    'Name(s) Found': player_this,
                    'ORIGINAL JERSEY': dataset_name.at[k, f'{column}'],
                    'Match Case': 'single' #single, multiple, none or cyrillic
                }        
                not_found_dataset = pd.concat([not_found_dataset, pd.DataFrame([data_row])], ignore_index=True)    
            else:
                print('DID NOT FIND ', player_this)
                counter += 1
                data_row = {
                    'Match ID': dataset_name.at[k, 'Match ID'],
                    'Competition': competition_this,
                    'Match': dataset_name.at[k, 'Match'],
                    'Date': date_this,
                    'Nationality': nationality_this,
                    'Team Country Code': country_code_this,
                    'Season': full_num,
                    'Status': 'FAIL',
                    'Name': player_this, #this will be zero if a fail happens 
                    'Market Value': 0,
                    'Lookup Still Required?': True,
                    'Lookup Return Case': 'Did Not Find in any Season', #Did Not Find in Curr Season, One Up, or One Down
                    'Impute Required?': False,
                    'Name(s) Found': player_this,
                    'ORIGINAL JERSEY': dataset_name.at[k, f'{column}'],
                    'Match Case': 'single' #single, multiple, none or cyrillic
                }        
                not_found_dataset = pd.concat([not_found_dataset, pd.DataFrame([data_row])], ignore_index=True)     
            
    #dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == country_code_this]['Name'].unique()
        #make the dataset nationality equal to only the season before , current or after 

print(counter)



0 Germany 2014-07-13
Manuel Neuer DE 2014
1 1 1
Mats Hummels DE 2014
1 1 1
Jérôme Boateng DE 2014
1 1 1
Benedikt Höwedes DE 2014
1 1 1
Philipp Lahm DE 2014
0 1 0
Bastian Schweinsteiger DE 2014
1 1 1
Christoph Kramer DE 2014
1 1 1
Toni Kroos DE 2014
1 1 1
Mesut Özil DE 2014
1 1 1
Thomas Müller DE 2014
1 1 1
Miroslav Klose DE 2014
0 1 0
1 Argentina 2014-07-13
Sergio Romero AR 2014
1 1 1
Ezequiel Garay AR 2014
1 1 1
Martín Demichelis AR 2014
1 1 1
Marcos Rojo AR 2014
1 1 1
Pablo Zabaleta AR 2014
1 1 1
Lucas Biglia AR 2014
1 1 1
Javier Mascherano AR 2014
1 1 1
Enzo Pérez AR 2014
1 1 1
Ezequiel Lavezzi AR 2014
1 1 1
Lionel Messi AR 2014
1 1 1
Gonzalo Higuaín AR 2014
1 1 1
2 Brazil 2014-07-12
Júlio César BR 2014
0 1 0
Thiago Silva BR 2014
1 1 1
David Luiz BR 2014
1 1 1
Maxwell BR 2014
0 1 0
Maicon BR 2014
1 1 0
Luiz Gustavo BR 2014
1 1 1
Paulinho BR 2014
0 1 1
Ramires BR 2014
1 1 0
Oscar BR 2014
1 1 1
Willian BR 2014
1 1 1
Jô BR 2014
0 1 0
3 Netherlands 2014-07-12
Jasper Cillessen NL 2014
1 

In [315]:
not_found_dataset['Status New'] = ''
for index,row in not_found_dataset.iterrows():
    if row['Status New'] != 'Done':
        #time.sleep(1)
        player_this = remove_prefix(row['ORIGINAL JERSEY'])
        if player_this[0] == ' ':
                player_this = player_this[1:]
        code = row['Team Country Code']
        full_num = row['Season']
        dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == code) & 
                                            (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
        print(player_this)
        if unidecode(player_this) in dataset_nationality:
            print(index, player_this, 'ya')
            number, reason = find_in_transfermarkt(unidecode(player_this), full_num, code, leagues_value)
            print(number, reason)
            if type(number) == float:
                0==0
            else:
                number = add_correct_money(number)  
            
            not_found_dataset.at[index, 'Market Value'] = number
            not_found_dataset.at[index, 'Lookup Return Case'] = reason
            not_found_dataset.at[index, 'Lookup Still Required?'] = False
            not_found_dataset.at[index, 'Status'] = 'SUCCESS'
            not_found_dataset.at[index, 'Status New'] = 'Done'

            # data_row = {
            #     'Match ID': row['Match ID'],
            #     'Competition': row['Competition'],
            #     'Match': row['Match'],
            #     'Date': row['Date'],
            #     'Nationality': row['Nationality'],
            #     'Team Country Code': row['Team Country Code'],
            #     'Season': row['Season'],
            #     'Status': 'SUCCESS',
            #     'Name': player_this, #this will be zero if a fail happens 
            #     'Market Value': number,
            #     'Lookup Still Required?': False,
            #     'Lookup Return Case': reason,
            #     'Impute Required?': False,
            #     'Name(s) Found': player_this,
            #     'ORIGINAL JERSEY': lineups_euros_qualifiers.at[k, f'{column}'],
            #     'Match Case': 'single' #single, multiple, none or cyrillic
            # }
            # lookup_dataset_1 = pd.concat([lookup_dataset_1, pd.DataFrame([data_row])], ignore_index=True)
        else:
            print(index, player_this, 'nein')
            if player_this in leagues_value[leagues_value['Team 1 Code'] == code]['Name'].unique():
                # not_found_dataset.at[index, 'Market Value'] = number
                not_found_dataset.at[index, 'Lookup Return Case'] = reason
                not_found_dataset.at[index, 'Lookup Still Required?'] = True
                not_found_dataset.at[index, 'Status'] = 'FAIL - NEW'
                not_found_dataset.at[index, 'Status New'] = 'Done'
                print('finished ', player_this)
    else:
        0==0
        print('Done Case')

Gökhan Inler
0 Gökhan Inler nein
finished  Gökhan Inler
Eric-Maxim Choupo-Moting
1 Eric-Maxim Choupo-Moting nein
finished  Eric-Maxim Choupo-Moting
Eric-Maxim Choupo-Moting
2 Eric-Maxim Choupo-Moting nein
finished  Eric-Maxim Choupo-Moting
Mario Mandžukić
3 Mario Mandžukić ya
Mario Mandzukic HR 2014
1 1 1
26000000.0 Found in Curr Season
Mario Mandžukić
4 Mario Mandžukić ya
Mario Mandzukic HR 2014
1 1 1
26000000.0 Found in Curr Season
Eric-Maxim Choupo-Moting
5 Eric-Maxim Choupo-Moting nein
finished  Eric-Maxim Choupo-Moting
Gökhan Inler
6 Gökhan Inler nein
finished  Gökhan Inler
Gökhan Inler
7 Gökhan Inler nein
finished  Gökhan Inler
Gökhan Inler
8 Gökhan Inler nein
finished  Gökhan Inler
Mario Mandžukić
9 Mario Mandžukić ya
Mario Mandzukic HR 2018
0 1 0
14400000.0 Only Before
Mario Mandžukić
10 Mario Mandžukić ya
Mario Mandzukic HR 2018
0 1 0
14400000.0 Only Before
Mario Mandžukić
11 Mario Mandžukić ya
Mario Mandzukic HR 2018
0 1 0
14400000.0 Only Before
Mario Mandžukić
12 Mario Mandž

In [ ]:
for index, row in not_found_dataset.iterrows():
    if row['Status New'] == '':
        not_found_dataset.at[index, 'Status'] = 'FAIL - NEW'

In [322]:
final_dataset_wc = pd.concat([lookup_dataset_1, not_found_dataset], ignore_index= True)

In [324]:
final_dataset_wc.to_csv('world_cup_lineups_data_complete.csv')

### Copa America

In [330]:
copa_america_lineups = load_csv_dataset('NEW LINEUPS to look up/Temp Lineup Data - Copa America Lineups.csv')

In [331]:
dataset_name = copa_america_lineups

In [332]:
dataset_name['Code'] = dataset_name['TeamName'].apply(get_country_code)

for i in range(0, len(dataset_name)):
    if dataset_name.at[i, 'Date'].startswith('Sat '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Sat ')[1]
    elif dataset_name.at[i, 'Date'].startswith('Sun '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Sun ')[1]
    elif dataset_name.at[i, 'Date'].startswith('Wed '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Wed ')[1]
    elif dataset_name.at[i, 'Date'].startswith('Tue '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Tue ')[1]
    elif dataset_name.at[i, 'Date'].startswith('Thu '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Thu ')[1]
    elif dataset_name.at[i, 'Date'].startswith('Mon '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Mon ')[1]
    elif dataset_name.at[i, 'Date'].startswith('Fri '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Fri ')[1]
dataset_name['Date'] = pd.to_datetime(dataset_name['Date'])

dataset_name['Status New'] = ''

#lineups_euros = lineups_euros.drop(columns={'Match ID'})
dataset_name['Match ID'] = ''
for index, row in dataset_name.iterrows():
    comp = row['Competition General']
    index_of_unnamed = dataset_name.at[index, 'Unnamed: 0']
    match_ID = f"{comp}_{index_of_unnamed}"
    dataset_name.at[index, 'Match ID'] = match_ID

lookup_dataset_1 = pd.DataFrame()


not_found_dataset = pd.DataFrame()


In [336]:
for index, row in dataset_name.iterrows():
    if row['TeamName'] in countries_codes['Country'].unique():
        0==0
    else:
        print(row['TeamName'])

In [338]:
counter = 0
salary_boolean = False
#Start from index 2
for k in range(0, len(dataset_name)): #len(lineups_euros_qualifiers)

    nationality_this = dataset_name.at[k, 'TeamName']
    competition_this = dataset_name.at[k, 'Competition']
    #nationality_this = lineups_data.at[k, 'Match']
    date_this = dataset_name.at[k, 'Date']
    country_code_this = dataset_name.at[k, 'Code']
    date_this = str(date_this)
    if ' 00:00:00' in date_this:
        date_this = date_this.split(' 00:00:00')[0]
    print(k, nationality_this, date_this)
    for i in range(2, 13): 
        number = -999
        reason = ''

        column = dataset_name.columns[i]
        player_this = remove_prefix(dataset_name.at[k, f'{column}'])
        if player_this[0] == ' ':
            player_this = player_this[1:]

        if player_this == '-':
            0==0
        else:
            #print(player_this)
            
            if '-' in date_this:
                if len(date_this.split("-")[2]) == 4:
                    yearstr = date_this.split("-")[2] #this is a 4 digit number
                    full_num = int(yearstr)
                elif len(date_this.split("-")[2]) == 2:
                    if len(date_this.split("-")[0]) == 4:
                        yearstr = date_this.split("-")[0]
                        full_num = int(yearstr)
                    else:
                        yearstr = date_this.split("-")[2] #this is a 2 digit number
                        full_num = int('20' + yearstr)
        dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == country_code_this) & 
                                        (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
        if player_this in dataset_nationality:
            0==0
            
            number, reason = find_in_transfermarkt(player_this, full_num, country_code_this, leagues_value)
            #print(player_this, reason, add_correct_money(number))

            #add a new row to lookup dataset 1 
            if type(number) == float:
                0==0
            else:
                number = add_correct_money(number)
            if i == 13:
                lineups_euros.at[k, 'Status New'] = 'Done'

            data_row = {
                'Match ID': dataset_name.at[k, 'Match ID'],
                'Competition': competition_this,
                'Match': dataset_name.at[k, 'Match'],
                'Date': date_this,
                'Nationality': nationality_this,
                'Team Country Code': country_code_this,
                'Season': full_num,
                'Status': 'SUCCESS',
                'Name': player_this, #this will be zero if a fail happens 
                'Market Value': number,
                'Lookup Still Required?': False,
                'Lookup Return Case': reason,
                'Impute Required?': False,
                'Name(s) Found': player_this,
                'ORIGINAL JERSEY': dataset_name.at[k, f'{column}'],
                'Match Case': 'single' #single, multiple, none or cyrillic
            }
            lookup_dataset_1 = pd.concat([lookup_dataset_1, pd.DataFrame([data_row])], ignore_index=True)
            
        else:
            print(player_this, 'nope')
            # counter += 1
            dataset_nationality_all_years = leagues_value[(leagues_value['Team 1 Code'] == country_code_this)]['Name'].unique()
            if player_this in dataset_nationality_all_years:
                
                print('***Found Him', player_this)
                data_row = {
                    'Match ID': dataset_name.at[k, 'Match ID'],
                    'Competition': competition_this,
                    'Match': dataset_name.at[k, 'Match'],
                    'Date': date_this,
                    'Nationality': nationality_this,
                    'Team Country Code': country_code_this,
                    'Season': full_num,
                    'Status': 'FAIL',
                    'Name': player_this, #this will be zero if a fail happens 
                    'Market Value': 0,
                    'Lookup Still Required?': True,
                    'Lookup Return Case': 'Did Not Find in Curr Season, One Up, or One Down', 
                    'Impute Required?': False,
                    'Name(s) Found': player_this,
                    'ORIGINAL JERSEY': dataset_name.at[k, f'{column}'],
                    'Match Case': 'single' #single, multiple, none or cyrillic
                }        
                not_found_dataset = pd.concat([not_found_dataset, pd.DataFrame([data_row])], ignore_index=True)    
            else:
                print('DID NOT FIND ', player_this)
                counter += 1
                data_row = {
                    'Match ID': dataset_name.at[k, 'Match ID'],
                    'Competition': competition_this,
                    'Match': dataset_name.at[k, 'Match'],
                    'Date': date_this,
                    'Nationality': nationality_this,
                    'Team Country Code': country_code_this,
                    'Season': full_num,
                    'Status': 'FAIL',
                    'Name': player_this, #this will be zero if a fail happens 
                    'Market Value': 0,
                    'Lookup Still Required?': True,
                    'Lookup Return Case': 'Did Not Find in any Season', #Did Not Find in Curr Season, One Up, or One Down
                    'Impute Required?': False,
                    'Name(s) Found': player_this,
                    'ORIGINAL JERSEY': dataset_name.at[k, f'{column}'],
                    'Match Case': 'single' #single, multiple, none or cyrillic
                }        
                not_found_dataset = pd.concat([not_found_dataset, pd.DataFrame([data_row])], ignore_index=True)     
            
    #dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == country_code_this]['Name'].unique()
        #make the dataset nationality equal to only the season before , current or after 

print(counter)



0 Chile 2015-07-04
Claudio Bravo CL 2015
1 1 1
Gary Medel CL 2015
1 1 1
Francisco Silva CL 2015
1 1 1
Jean Beausejour CL 2015
1 1 1
Mauricio Isla CL 2015
1 1 1
Marcelo Díaz CL 2015
1 1 1
Arturo Vidal CL 2015
1 1 1
Charles Aránguiz CL 2015
1 1 1
Jorge Valdivia CL 2015
1 1 0
Alexis Sánchez CL 2015
1 1 1
Eduardo Vargas CL 2015
1 1 1
1 Argentina 2015-07-04
Sergio Romero AR 2015
1 1 1
Martín Demichelis AR 2015
1 1 0
Nicolás Otamendi AR 2015
1 1 1
Marcos Rojo AR 2015
1 1 0
Pablo Zabaleta AR 2015
1 1 0
Javier Mascherano AR 2015
1 1 0
Lucas Biglia AR 2015
1 1 1
Javier Pastore AR 2015
1 1 0
Ángel Di María AR 2015
1 1 1
Lionel Messi AR 2015
1 1 1
Sergio Agüero AR 2015
1 1 0
2 Peru 2015-07-03
Pedro Gallese PE 2015
1 1 1
Christian Ramos PE 2015
1 1 1
Carlos Ascues PE 2015
1 1 0
Juan Manuel Vargas PE 2015
1 1 0
Luis Advíncula PE 2015
1 1 1
Josepmir Ballón PE 2015
1 1 0
Carlos Lobatón PE 2015
1 1 0
Yordy Reyna PE 2015
1 1 0
Christian Cueva PE 2015
1 1 1
André Carrillo PE 2015
1 1 1
Paolo Guerrero PE

In [340]:
not_found_dataset['Status New'] = ''
for index, row in not_found_dataset.iterrows():
    if row['Status New'] == '':
        not_found_dataset.at[index, 'Status'] = 'FAIL - NEW'

In [341]:
not_found_dataset

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Status New
0,Copa America_28,Copa America 2016,USA vs Argentina,2016-06-21,USA,US,2016,FAIL - NEW,John Brooks,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,John Brooks,6 John Brooks,single,
1,Copa America_30,Copa America 2016,USA vs Ecuador,2016-06-16,USA,US,2016,FAIL - NEW,John Brooks,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,John Brooks,6 John Brooks,single,
2,Copa America_34,Copa America 2016,USA vs Colombia,2016-06-03,USA,US,2016,FAIL - NEW,John Brooks,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,John Brooks,6 John Brooks,single,
3,Copa America_36,Copa America 2016,USA vs Costa Rica,2016-06-07,USA,US,2016,FAIL - NEW,John Brooks,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,John Brooks,6 John Brooks,single,
4,Copa America_38,Copa America 2016,USA vs Paraguay,2016-06-11,USA,US,2016,FAIL - NEW,John Brooks,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,John Brooks,6 John Brooks,single,


In [343]:
final_dataset_copa_america = pd.concat([lookup_dataset_1, not_found_dataset], ignore_index=True)

In [345]:
final_dataset_copa_america.to_csv('copa_america_lineups_data_complete.csv')

### Nations League

In [374]:
nations_league_lineups = load_csv_dataset('NEW LINEUPS to look up/Temp Lineup Data - UEFA Nations League Lineups (1).csv')

In [375]:
dataset_name = nations_league_lineups

In [376]:
dataset_name['Code'] = dataset_name['TeamName'].apply(get_country_code)

for i in range(0, len(dataset_name)):
    if dataset_name.at[i, 'Date'].startswith('Sat '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Sat ')[1]
    elif dataset_name.at[i, 'Date'].startswith('Sun '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Sun ')[1]
    elif dataset_name.at[i, 'Date'].startswith('Wed '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Wed ')[1]
    elif dataset_name.at[i, 'Date'].startswith('Tue '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Tue ')[1]
    elif dataset_name.at[i, 'Date'].startswith('Thu '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Thu ')[1]
    elif dataset_name.at[i, 'Date'].startswith('Mon '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Mon ')[1]
    elif dataset_name.at[i, 'Date'].startswith('Fri '):
        dataset_name.at[i, 'Date'] = dataset_name.at[i, 'Date'].split('Fri ')[1]
dataset_name['Date'] = pd.to_datetime(dataset_name['Date'])

dataset_name['Status New'] = ''

#lineups_euros = lineups_euros.drop(columns={'Match ID'})
dataset_name['Match ID'] = ''
for index, row in dataset_name.iterrows():
    comp = row['Competition General']
    index_of_unnamed = dataset_name.at[index, 'Unnamed: 0']
    match_ID = f"{comp}_{index_of_unnamed}"
    dataset_name.at[index, 'Match ID'] = match_ID

lookup_dataset_1 = pd.DataFrame()


not_found_dataset = pd.DataFrame()


In [377]:
#Euros Qualifiers
#Macedonia --> North Macedonia
#Czechia --> Czech Republic
#N. Ireland --> Northern Ireland
for index, row in dataset_name.iterrows():
    if row['TeamName'] == 'Macedonia':
        dataset_name.at[index, 'TeamName'] = 'North Macedonia'
        dataset_name.at[index, 'Match'] = dataset_name.at[index, 'Match'].replace('Macedonia', 'North Macedonia')
    if row['TeamName'] == 'Czechia':
        dataset_name.at[index, 'TeamName'] = 'Czech Republic'
        dataset_name.at[index, 'Match'] = dataset_name.at[index, 'Match'].replace('Czechia', 'Czech Republic')
    if row['TeamName'] == 'N. Ireland':
        dataset_name.at[index, 'TeamName'] = 'Northern Ireland'
        dataset_name.at[index, 'Match'] = dataset_name.at[index, 'Match'].replace('N. Ireland', 'Northern Ireland')
dataset_name['Code'] = dataset_name['TeamName'].apply(get_country_code)


In [378]:
for index, row in dataset_name.iterrows():
    if row['TeamName'] in countries_codes['Country'].unique():
        0==0
    else:
        print(row['TeamName'])

In [379]:
counter = 0
salary_boolean = False
#Start from index 2
for k in range(0, len(dataset_name)): #len(lineups_euros_qualifiers)

    nationality_this = dataset_name.at[k, 'TeamName']
    competition_this = dataset_name.at[k, 'Competition']
    #nationality_this = lineups_data.at[k, 'Match']
    date_this = dataset_name.at[k, 'Date']
    country_code_this = dataset_name.at[k, 'Code']
    date_this = str(date_this)
    if ' 00:00:00' in date_this:
        date_this = date_this.split(' 00:00:00')[0]
    print(k, nationality_this, date_this)
    for i in range(2, 13): 
        number = -999
        reason = ''

        column = dataset_name.columns[i]
        player_this = remove_prefix(dataset_name.at[k, f'{column}'])
        if player_this == '-':
            0==0
        elif player_this == '':
            0==0
        else:
            if player_this[0] == ' ':
                player_this = player_this[1:]

        if player_this == '-':
            0==0
        else:
            #print(player_this)
            
            if '-' in date_this:
                if len(date_this.split("-")[2]) == 4:
                    yearstr = date_this.split("-")[2] #this is a 4 digit number
                    full_num = int(yearstr)
                elif len(date_this.split("-")[2]) == 2:
                    if len(date_this.split("-")[0]) == 4:
                        yearstr = date_this.split("-")[0]
                        full_num = int(yearstr)
                    else:
                        yearstr = date_this.split("-")[2] #this is a 2 digit number
                        full_num = int('20' + yearstr)
        dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == country_code_this) & 
                                        (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
        if player_this in dataset_nationality:
            0==0
            
            number, reason = find_in_transfermarkt(player_this, full_num, country_code_this, leagues_value)
            #print(player_this, reason, add_correct_money(number))

            #add a new row to lookup dataset 1 
            if type(number) == float:
                0==0
            else:
                number = add_correct_money(number)
            if i == 13:
                lineups_euros.at[k, 'Status New'] = 'Done'

            data_row = {
                'Match ID': dataset_name.at[k, 'Match ID'],
                'Competition': competition_this,
                'Match': dataset_name.at[k, 'Match'],
                'Date': date_this,
                'Nationality': nationality_this,
                'Team Country Code': country_code_this,
                'Season': full_num,
                'Status': 'SUCCESS',
                'Name': player_this, #this will be zero if a fail happens 
                'Market Value': number,
                'Lookup Still Required?': False,
                'Lookup Return Case': reason,
                'Impute Required?': False,
                'Name(s) Found': player_this,
                'ORIGINAL JERSEY': dataset_name.at[k, f'{column}'],
                'Match Case': 'single' #single, multiple, none or cyrillic
            }
            lookup_dataset_1 = pd.concat([lookup_dataset_1, pd.DataFrame([data_row])], ignore_index=True)
            
        else:
            print(player_this, 'nope')
            # counter += 1
            dataset_nationality_all_years = leagues_value[(leagues_value['Team 1 Code'] == country_code_this)]['Name'].unique()
            if player_this in dataset_nationality_all_years:
                
                print('***Found Him', player_this)
                data_row = {
                    'Match ID': dataset_name.at[k, 'Match ID'],
                    'Competition': competition_this,
                    'Match': dataset_name.at[k, 'Match'],
                    'Date': date_this,
                    'Nationality': nationality_this,
                    'Team Country Code': country_code_this,
                    'Season': full_num,
                    'Status': 'FAIL',
                    'Name': player_this, #this will be zero if a fail happens 
                    'Market Value': 0,
                    'Lookup Still Required?': True,
                    'Lookup Return Case': 'Did Not Find in Curr Season, One Up, or One Down', 
                    'Impute Required?': False,
                    'Name(s) Found': player_this,
                    'ORIGINAL JERSEY': dataset_name.at[k, f'{column}'],
                    'Match Case': 'single' #single, multiple, none or cyrillic
                }        
                not_found_dataset = pd.concat([not_found_dataset, pd.DataFrame([data_row])], ignore_index=True)    
            else:
                print('DID NOT FIND ', player_this)
                counter += 1
                data_row = {
                    'Match ID': dataset_name.at[k, 'Match ID'],
                    'Competition': competition_this,
                    'Match': dataset_name.at[k, 'Match'],
                    'Date': date_this,
                    'Nationality': nationality_this,
                    'Team Country Code': country_code_this,
                    'Season': full_num,
                    'Status': 'FAIL',
                    'Name': player_this, #this will be zero if a fail happens 
                    'Market Value': 0,
                    'Lookup Still Required?': True,
                    'Lookup Return Case': 'Did Not Find in any Season', #Did Not Find in Curr Season, One Up, or One Down
                    'Impute Required?': False,
                    'Name(s) Found': player_this,
                    'ORIGINAL JERSEY': dataset_name.at[k, f'{column}'],
                    'Match Case': 'single' #single, multiple, none or cyrillic
                }        
                not_found_dataset = pd.concat([not_found_dataset, pd.DataFrame([data_row])], ignore_index=True)     
            
    #dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == country_code_this]['Name'].unique()
        #make the dataset nationality equal to only the season before , current or after 

print(counter)



0 Germany 2018-09-06
Manuel Neuer DE 2018
1 1 1
Mats Hummels DE 2018
1 1 0
Jérôme Boateng DE 2018
1 1 0
Antonio Rüdiger DE 2018
1 1 0
Matthias Ginter DE 2018
1 1 1
Joshua Kimmich DE 2018
1 1 1
Toni Kroos DE 2018
1 1 1
Leon Goretzka DE 2018
1 1 1
Timo Werner DE 2018
1 1 1
Thomas Müller DE 2018
1 1 0
Marco Reus DE 2018
1 1 1
1 France 2018-09-06
Alphonse Areola FR 2018
1 1 1
Samuel Umtiti FR 2018
1 1 0
Raphaël Varane FR 2018
1 1 1
Lucas Hernández FR 2018
1 1 0
Benjamin Pavard FR 2018
1 1 1
N'Golo Kanté FR 2018
1 1 1
Paul Pogba FR 2018
1 1 0
Blaise Matuidi FR 2018
1 1 0
Kylian Mbappé FR 2018
1 1 1
Antoine Griezmann FR 2018
1 1 1
Olivier Giroud FR 2018
1 1 1
2 France 2018-09-09
Alphonse Areola FR 2018
1 1 1
Samuel Umtiti FR 2018
1 1 0
Raphaël Varane FR 2018
1 1 1
Lucas Hernández FR 2018
1 1 0
Benjamin Pavard FR 2018
1 1 1
N'Golo Kanté FR 2018
1 1 1
Paul Pogba FR 2018
1 1 0
Blaise Matuidi FR 2018
1 1 0
Kylian Mbappé FR 2018
1 1 1
Antoine Griezmann FR 2018
1 1 1
Olivier Giroud FR 2018
1 1 1
3

In [380]:
not_found_dataset

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case
0,Nations League_12,Nations League 2018-2020,Italy vs Poland,2018-09-07,Poland,PL,2018,FAIL,Bartosz Bereszyński,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Bartosz Bereszyński,18 Bartosz Bereszyński,single
1,Nations League_14,Nations League 2018-2020,Poland vs Portugal,2018-10-11,Poland,PL,2018,FAIL,Bartosz Bereszyński,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Bartosz Bereszyński,18 Bartosz Bereszyński,single
2,Nations League_15,Nations League 2018-2020,Poland vs Italy,2018-10-14,Poland,PL,2018,FAIL,Wojciech Szczęsny,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Wojciech Szczęsny,1 Wojciech Szczęsny,single
3,Nations League_15,Nations League 2018-2020,Poland vs Italy,2018-10-14,Poland,PL,2018,FAIL,Bartosz Bereszyński,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Bartosz Bereszyński,18 Bartosz Bereszyński,single
4,Nations League_17,Nations League 2018-2020,Portugal vs Poland,2018-11-20,Poland,PL,2018,FAIL,Wojciech Szczęsny,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Wojciech Szczęsny,1 Wojciech Szczęsny,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,Nations League_441,Nations League 2022-2023,Andorra vs Moldova,2022-06-06,Andorra,AD,2022,FAIL,Josep Gómes,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Josep Gómes,1 Josep Gómes,single
255,Nations League_441,Nations League 2022-2023,Andorra vs Moldova,2022-06-06,Moldova,MD,2022,FAIL,Dorian Railean,0,True,Did Not Find in any Season,False,Dorian Railean,23 Dorian Railean,single
256,Nations League_442,Nations League 2022-2023,Moldova vs Latvia,2022-06-10,Moldova,MD,2022,FAIL,Dorian Railean,0,True,Did Not Find in any Season,False,Dorian Railean,23 Dorian Railean,single
257,Nations League_443,Nations League 2022-2023,Andorra vs Liechtenstein,2022-06-10,Andorra,AD,2022,FAIL,Josep Gómes,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Josep Gómes,1 Josep Gómes,single


In [384]:
not_found_dataset['Status New'] = ''
for index, row in not_found_dataset.iterrows():
    if row['ORIGINAL JERSEY'] == '-':
        not_found_dataset.at[index, 'Status New'] = 'Done'
        not_found_dataset.at[index, 'Status'] = 'FAIL - EMPTY'
        

In [385]:
for index,row in not_found_dataset.iterrows():
    if row['Status New'] != 'Done':
        #time.sleep(1)
        if row['ORIGINAL JERSEY'] != '':
            player_this = remove_prefix(row['ORIGINAL JERSEY'])
            if player_this[0] == ' ':
                    player_this = player_this[1:]
            code = row['Team Country Code']
            full_num = row['Season']
            dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == code) & 
                                                (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
            print(player_this)
            if unidecode(player_this) in dataset_nationality:
                print(index, player_this, 'ya')
                number, reason = find_in_transfermarkt(unidecode(player_this), full_num, code, leagues_value)
                print(number, reason)
                if type(number) == float:
                    0==0
                else:
                    number = add_correct_money(number)  
                
                not_found_dataset.at[index, 'Market Value'] = number
                not_found_dataset.at[index, 'Lookup Return Case'] = reason
                not_found_dataset.at[index, 'Lookup Still Required?'] = False
                not_found_dataset.at[index, 'Status'] = 'SUCCESS'
                not_found_dataset.at[index, 'Status New'] = 'Done'

                # data_row = {
                #     'Match ID': row['Match ID'],
                #     'Competition': row['Competition'],
                #     'Match': row['Match'],
                #     'Date': row['Date'],
                #     'Nationality': row['Nationality'],
                #     'Team Country Code': row['Team Country Code'],
                #     'Season': row['Season'],
                #     'Status': 'SUCCESS',
                #     'Name': player_this, #this will be zero if a fail happens 
                #     'Market Value': number,
                #     'Lookup Still Required?': False,
                #     'Lookup Return Case': reason,
                #     'Impute Required?': False,
                #     'Name(s) Found': player_this,
                #     'ORIGINAL JERSEY': lineups_euros_qualifiers.at[k, f'{column}'],
                #     'Match Case': 'single' #single, multiple, none or cyrillic
                # }
                # lookup_dataset_1 = pd.concat([lookup_dataset_1, pd.DataFrame([data_row])], ignore_index=True)
            else:
                print(index, player_this, 'nein')
                if player_this in leagues_value[leagues_value['Team 1 Code'] == code]['Name'].unique():
                    # not_found_dataset.at[index, 'Market Value'] = number
                    not_found_dataset.at[index, 'Lookup Return Case'] = reason
                    not_found_dataset.at[index, 'Lookup Still Required?'] = True
                    not_found_dataset.at[index, 'Status'] = 'FAIL - NEW'
                    not_found_dataset.at[index, 'Status New'] = 'Done'
                    print('finished ', player_this)
        else:
            print('null row')
    else:
        0==0
        print('Done Case')

Bartosz Bereszyński
0 Bartosz Bereszyński ya
Bartosz Bereszynski PL 2018
1 1 0
10000000.0 Found in Curr Season
Bartosz Bereszyński
1 Bartosz Bereszyński ya
Bartosz Bereszynski PL 2018
1 1 0
10000000.0 Found in Curr Season
Wojciech Szczęsny
2 Wojciech Szczęsny ya
Wojciech Szczesny PL 2018
1 1 1
35000000.0 Found in Curr Season
Bartosz Bereszyński
3 Bartosz Bereszyński ya
Bartosz Bereszynski PL 2018
1 1 0
10000000.0 Found in Curr Season
Wojciech Szczęsny
4 Wojciech Szczęsny ya
Wojciech Szczesny PL 2018
1 1 1
35000000.0 Found in Curr Season
Bartosz Bereszyński
5 Bartosz Bereszyński ya
Bartosz Bereszynski PL 2018
1 1 0
10000000.0 Found in Curr Season
Vladimír Weiss
6 Vladimír Weiss ya
Vladimir Weiss SK 2018
1 0 0
2500000.0 Found in Curr Season
Róbert Mak
7 Róbert Mak ya
Robert Mak SK 2018
1 1 1
4500000.0 Found in Curr Season
Peter Pekarík
8 Peter Pekarík ya
Peter Pekarik SK 2018
1 1 1
800000.0 Found in Curr Season
Róbert Mak
9 Róbert Mak ya
Robert Mak SK 2018
1 1 1
4500000.0 Found in Curr S

In [390]:
not_found_dataset[not_found_dataset['Status'] == 'FAIL - EMPTY']['Name(s) Found'].value_counts()

Name(s) Found
    44
Name: count, dtype: int64

In [395]:
final_dataset_nations_league = pd.concat([lookup_dataset_1, not_found_dataset], ignore_index=True)
final_dataset_nations_league

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Status New
0,Nations League_0,Nations League 2018-2020,Germany vs France,2018-09-06,Germany,DE,2018,SUCCESS,Manuel Neuer,18000000.0,False,Found in Curr Season,False,Manuel Neuer,1 Manuel Neuer,single,NaN
1,Nations League_0,Nations League 2018-2020,Germany vs France,2018-09-06,Germany,DE,2018,SUCCESS,Mats Hummels,35000000.0,False,Found in Curr Season,False,Mats Hummels,5 Mats Hummels,single,NaN
2,Nations League_0,Nations League 2018-2020,Germany vs France,2018-09-06,Germany,DE,2018,SUCCESS,Jérôme Boateng,20000000.0,False,Found in Curr Season,False,Jérôme Boateng,17 Jérôme Boateng,single,NaN
3,Nations League_0,Nations League 2018-2020,Germany vs France,2018-09-06,Germany,DE,2018,SUCCESS,Antonio Rüdiger,50000000.0,False,Found in Curr Season,False,Antonio Rüdiger,16 Antonio Rüdiger,single,NaN
4,Nations League_0,Nations League 2018-2020,Germany vs France,2018-09-06,Germany,DE,2018,SUCCESS,Matthias Ginter,35000000.0,False,Found in Curr Season,False,Matthias Ginter,4 Matthias Ginter,single,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10027,Nations League_441,Nations League 2022-2023,Andorra vs Moldova,2022-06-06,Andorra,AD,2022,SUCCESS,Josep Gómes,10000.0,False,Found in Curr Season,False,Josep Gómes,1 Josep Gómes,single,Done
10028,Nations League_441,Nations League 2022-2023,Andorra vs Moldova,2022-06-06,Moldova,MD,2022,FAIL,Dorian Railean,0.0,True,Did Not Find in any Season,False,Dorian Railean,23 Dorian Railean,single,
10029,Nations League_442,Nations League 2022-2023,Moldova vs Latvia,2022-06-10,Moldova,MD,2022,FAIL,Dorian Railean,0.0,True,Did Not Find in any Season,False,Dorian Railean,23 Dorian Railean,single,
10030,Nations League_443,Nations League 2022-2023,Andorra vs Liechtenstein,2022-06-10,Andorra,AD,2022,SUCCESS,Josep Gómes,10000.0,False,Found in Curr Season,False,Josep Gómes,1 Josep Gómes,single,Done


In [396]:
final_dataset_nations_league.to_csv('nations_league_lineups_data_complete.csv')

## Combined DF - old

In [57]:
len(combined_df)

56928

In [58]:
old_df = load_csv_dataset('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/combined_df_may_10.csv')

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_59433/2775728346.py:54: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [61]:
pd.DataFrame(old_df['Competition'].value_counts())

,count
Competition,
"World Cup Qualifiers, Europe, 2018",6079
"Euros 2020, Qualifiers",5740
"World Cup Qualifiers, Europe, 2022",5646
"Euros 2024, Qualifiers",5012
"World Cup Qualifiers, Asia, 2022",4763
"World Cup Qualifiers, Asia, 2018",4752
"Euros 2016, Qualifiers",4168
"World Cup Qualifiers, Europe, 2014",3666
Nations League 2020-2021,3590
